# EXPERIMENTACIÓN:

In [13]:
# ============================================================
#  PLN - Proyecto Tweets Aseguradoras (Módulo 1: Léxico)
#  Versión ajustada para mejorar detección de marcas
# ============================================================

# --------- 0) Setup de entorno (instala si falta) ----------
import sys, subprocess, importlib

def ensure(pkg, pip_name=None):
    try:
        importlib.import_module(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name or pkg])

# Librerías base
ensure("pandas")
ensure("numpy", "numpy<2.0")
ensure("regex")
ensure("unidecode")
ensure("nltk")
ensure("sklearn", "scikit-learn")

import pandas as pd
import numpy as np
import regex as re
from unidecode import unidecode
import nltk
nltk.download('punkt', quiet=True)
from nltk.stem.snowball import SpanishStemmer

# spaCy (modelo español)
ensure("spacy")
import spacy
try:
    nlp = spacy.load("es_core_news_sm")
except:
    subprocess.check_call([sys.executable, "-m", "spacy", "download", "es_core_news_sm"])
    nlp = spacy.load("es_core_news_sm")

# --------- 1) Parámetros de proyecto -----------------------
EXCEL_PATH = r"C:\Users\santi\Downloads\Learning\Maestria\Programacion Lenguaje Natural\Proyecto\Inputs\2053806800_Aseguradoras+Query.xlsx"
SHEET_NAME = "Hoja1"

# Columnas esperadas (ajusta si varían)
COL_DATE   = "Date"
COL_TITLE  = "Title"
COL_SNIPP  = "Snippet"
COL_URL    = "Url"
COL_DOMAIN = "Domain"
COL_LANG   = "Language"
COL_COUNTRY = "Country"
TEXT_COL   = "text_raw"

# MWE (frases fijas)
MWE_SERVICE = [
    "atención al cliente", "pago de siniestro", "tiempo de respuesta",
    "radicado de pqrs", "red médica", "autorización de servicio",
    "incremento de prima", "línea de atención"
]

MWE_ENTITIES = [
    "grupo sura", "fondo de pensiones obligatorias"
]

# Gazetteer de marcas
BRANDS = {
    "seguros bolivar": {"seguros bolívar", "seguros bolivar", "@segurosbolivar",
                        "grupobolivar", "grupo bolívar", "grupo bolivar", "segurosbolivar"},
    "sura": {"sura", "grupo sura", "@suramericana", "suramericana"},
    "axa colpatria": {"axa colpatria", "@axacolpatria", "colpatria seguros","axacolpatria"},
    "mapfre": {"mapfre", "@mapfre_col","mapfre_co"},
    "allianz": {"allianz", "@allianzcol"},
    "liberty seguros": {"liberty seguros", "@libertycol"},
    "zurich": {"zurich", "@zurichcolombia"},
    "hdi seguros": {"hdi seguros", "@hdi_segurosco"},
    "seguros del estado": {"seguros del estado", "@segurosdelestado"},
    "porvenir": {"porvenir", "@porvenir", "fondo de pensiones obligatorias porvenir"},
    "Fiduciaria La Previsora": {"fiduciaria la previsora", "@fiduciarialaprevisora",
                                "fiduciaria la previsora","previsora","la previsora s.a. compañía de seguros"},
    "Positiva Compañía de Seguros": {"positiva compañía de seguros", "positivacol"},
}

# Mapping dominio -> marca
DOMAIN2BRAND = {
    "segurosbolivar.com": "seguros bolivar",
    "grupobolivar.com": "seguros bolivar",
    "sura.com": "sura",
    "segurossura.com": "sura",
    "axacolpatria.co": "axa colpatria",
    "mapfre.com": "mapfre",
    "allianz.co": "allianz",
    "libertycolombia.com": "liberty seguros",
    "zurich.com.co": "zurich",
    "hdi.com.co": "hdi seguros",
    "segurosdelestado.com": "seguros del estado",
    "porvenir.com.co": "porvenir",
}

# Stopwords de dominio
DOMAIN_STOPWORDS = {
    "seguros", "aseguradora", "aseguradoras", "póliza", "poliza", "asegurado", "asegurada",
    *set(DOMAIN2BRAND.values()), *set(BRANDS.keys()),
}

# --------- 2) Normalización ------------------
URL_RE = re.compile(r"https?://\S+|www\.\S+", flags=re.IGNORECASE)
MENTION_RE = re.compile(r"@\w+", flags=re.IGNORECASE)
HASHTAG_RE = re.compile(r"#\w+", flags=re.IGNORECASE)
MULTISPACE_RE = re.compile(r"\s+")

def basic_clean(text: str) -> str:
    """Limpieza robusta: conserva menciones y hashtags como texto legible."""
    if not isinstance(text, str):
        return ""
    t = text

    # ⚙️ Guardar dominios antes de eliminar URLs
    urls = re.findall(URL_RE, t)
    t = URL_RE.sub(" ", t)

    # ⚙️ Mantener menciones sin @ (para que 'segurosbolivar' siga apareciendo)
    t = MENTION_RE.sub(lambda m: " " + m.group(0)[1:] + " ", t)

    # ⚙️ Conservar hashtags como palabras (sin #)
    t = HASHTAG_RE.sub(lambda m: " " + m.group(0)[1:] + " ", t)

    # Limpieza general
    t = t.replace("’", "'").replace("“", '"').replace("”", '"')
    t = t.lower()
    t = MULTISPACE_RE.sub(" ", t).strip()

    # ⚙️ Añadir URLs al final para detección de dominio
    if urls:
        t += " " + " ".join(urls)
    return t


def ascii_fold(text: str) -> str:
    """Quita tildes y normaliza a minúsculas."""
    return unidecode(text or "").lower()

def block_mwes(text: str, mwe_list) -> str:
    """Reemplaza espacios en MWE por '_' para tratarlas como tokens únicos."""
    t = " " + text + " "
    for phrase in sorted(mwe_list, key=len, reverse=True):
        p_norm = ascii_fold(phrase)
        pattern = re.escape(p_norm).replace(r"\ ", r"\s+")
        rx = re.compile(rf"(?i)(?<!\w){pattern}(?!\w)")
        t_norm = ascii_fold(t)
        t_norm = rx.sub(phrase.replace(" ", "_"), t_norm)
        t = t_norm
    return t.strip()

# --------- 3) Extracción de marcas -------------------------
def extract_brands(text_norm: str, domain: str = "") -> list:
    """Detección flexible de marcas: menciones pegadas, hashtags y dominios."""
    found = set()
    t = ascii_fold(text_norm)

    # Búsqueda por texto
    for brand, aliases in BRANDS.items():
        for alias in aliases | {brand}:
            alias_norm = ascii_fold(alias)
            # ⚙️ patrón flexible: detecta alias pegados (#, @, sin espacios)
            pattern = rf"(?<!\w){re.escape(alias_norm)}(?!\w)"
            if re.search(pattern, t):
                found.add(brand)
                break

    # Búsqueda por dominio (URL o columna)
    d = (domain or "").lower().strip()
    d = d.replace("https://", "").replace("http://", "")
    d = d.split("/")[0] if d else d
    for suf, bname in DOMAIN2BRAND.items():
        if suf in t or d.endswith(suf):
            found.add(bname)

    return sorted(found)

def pick_primary_brand(brands: list, text_norm: str) -> str:
    """Escoge la marca principal por posición de aparición."""
    if not brands:
        return ""
    t = ascii_fold(text_norm)
    positions = []
    for b in brands:
        aliases = list(BRANDS.get(b, [])) + [b]
        pos_min = min([t.find(ascii_fold(a)) for a in aliases if ascii_fold(a) in t] or [10**9])
        positions.append((pos_min, b))
    positions.sort(key=lambda x: x[0])
    return positions[0][1] if positions[0][0] < 10**9 else sorted(brands)[0]

# --------- 4) Tokenización -------------------
stemmer = SpanishStemmer()

def tokenize_lemma(text_blocked: str) -> list:
    doc = nlp(text_blocked)
    toks = [tok.lemma_ for tok in doc if not tok.is_space and not tok.is_punct]
    return [t for t in toks if t]

def tokenize_stem(text_blocked: str) -> list:
    doc = nlp(text_blocked)
    toks = [stemmer.stem(tok.text.strip()) for tok in doc if not tok.is_space and not tok.is_punct]
    return [t for t in toks if t]

# --------- 5) Métricas EDA -------------------
def ttr(tokens: list) -> float:
    return (len(set(tokens)) / max(1, len(tokens))) if tokens else 0.0

def hapax_ratio(tokens: list) -> float:
    if not tokens:
        return 0.0
    from collections import Counter
    c = Counter(tokens)
    hapax = sum(1 for _, f in c.items() if f == 1)
    return hapax / len(c)

# ============================================================
#  A) CARGA DE DATOS
# ============================================================
df = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_NAME)
if "id" not in df.columns.str.lower():
    df.insert(0, "id", range(1, len(df)+1))

df[TEXT_COL] = (df.get(COL_TITLE, "").fillna("").astype(str) + " " +
                df.get(COL_SNIPP, "").fillna("").astype(str)).str.strip()

if COL_LANG in df.columns:
    df = df[df[COL_LANG].str.lower().isin(["es", "español"])]

df = df.reset_index(drop=True)
print(f"Registros cargados: {len(df)}")

# ============================================================
#  B) LIMPIEZA + MWE
# ============================================================
df["text_clean_base"] = df[TEXT_COL].map(basic_clean)
df["text_mwe_entities"] = df["text_clean_base"].map(lambda t: block_mwes(t, MWE_ENTITIES))
df["text_blocked"] = df["text_mwe_entities"].map(lambda t: block_mwes(t, MWE_SERVICE))

# ============================================================
#  C) EXTRACCIÓN DE MARCAS
# ============================================================
df["brand_list"] = df.apply(lambda r: extract_brands(r["text_blocked"], r.get(COL_DOMAIN, "")), axis=1)
df["brand_primary"] = df.apply(lambda r: pick_primary_brand(r["brand_list"], r["text_blocked"]), axis=1)

# ============================================================
#  D) TOKENIZACIÓN + TEXTOS
# ============================================================
df["tokens_lemma"] = df["text_blocked"].map(tokenize_lemma)
df["tokens_stem"]  = df["text_blocked"].map(tokenize_stem)

def remove_domain_words(tokens: list) -> list:
    return [t for t in tokens if t not in DOMAIN_STOPWORDS]

df["tokens_topics"] = df["tokens_lemma"].map(remove_domain_words)
df["text_lemma"]  = df["tokens_lemma"].map(lambda xs: " ".join(xs))
df["text_stem"]   = df["tokens_stem"].map(lambda xs: " ".join(xs))
df["text_topics"] = df["tokens_topics"].map(lambda xs: " ".join(xs))

# ============================================================
#  E) MÉTRICAS EDA
# ============================================================
df["n_chars_raw"]    = df[TEXT_COL].str.len().fillna(0).astype(int)
df["n_tokens_lemma"] = df["tokens_lemma"].map(len)
df["n_tokens_topics"]= df["tokens_topics"].map(len)
df["ttr_lemma"]      = df["tokens_lemma"].map(ttr)
df["hapax_lemma"]    = df["tokens_lemma"].map(hapax_ratio)

eda_by_brand = (
    df.groupby("brand_primary", dropna=False)
      .agg(n_docs=("id","count"),
           mean_len=("n_tokens_lemma","mean"),
           mean_ttr=("ttr_lemma","mean"))
      .sort_values("n_docs", ascending=False)
)
print(eda_by_brand.head(10))

# ============================================================
#  F) EXPORT "CONJUNTO MEDIDO"
# ============================================================
cols_measured = [
    "id", COL_DATE, COL_DOMAIN, COL_URL,
    TEXT_COL, "text_clean_base", "text_blocked",
    "brand_list", "brand_primary",
    "text_lemma", "text_stem", "text_topics",
    "n_chars_raw", "n_tokens_lemma", "n_tokens_topics",
    "ttr_lemma", "hapax_lemma",
]
export_df = df[[c for c in cols_measured if c in df.columns]].copy()

OUTPUT_CSV = "corpus_medido2.csv"
OUTPUT_PQ  = "corpus_medido2.parquet"

export_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
try:
    export_df.to_parquet(OUTPUT_PQ, index=False)
except Exception as e:
    print("Parquet no disponible -> solo CSV. Error:", e)

print(f"Exportado: {OUTPUT_CSV} (y parquet si estuvo disponible)")

Registros cargados: 15469
                              n_docs   mean_len  mean_ttr
brand_primary                                            
                                4987  53.885101  0.627808
seguros bolivar                 3379  56.077242  0.557132
Fiduciaria La Previsora         1915  59.916971  0.582862
sura                            1702  56.796122  0.631199
mapfre                          1187  55.144061  0.636045
Positiva Compañía de Seguros     933  58.722401  0.452000
axa colpatria                    663  56.457014  0.578748
allianz                          316  53.050633  0.686732
liberty seguros                  186  54.258065  0.690902
hdi seguros                      109  59.155963  0.659051
Exportado: corpus_medido2.csv (y parquet si estuvo disponible)


In [14]:
# ============================
# TOP N-GRAMAS (bigramas/trigramas)
# ============================
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

# --- Stopwords español (generales) ---
import nltk
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
sw_es = set(stopwords.words('spanish'))

# Si quieres quitar también tus stopwords de dominio en TEMAS:
from collections import Counter
# DOMAIN_STOPWORDS viene del bloque anterior (marcas, 'seguros', etc.)
sw_temas = sw_es.union(DOMAIN_STOPWORDS)

def top_ngrams(texts, ngram_range=(2,2), top_k=50, stopwords_set=None, min_df=2):
    """
    Devuelve un DataFrame con los top n-gramas por frecuencia absoluta.
    - texts: iterable de strings
    - ngram_range: (2,2) bigramas, (3,3) trigramas, etc.
    - stopwords_set: set de stopwords a remover
    - min_df: descarta n-gramas que aparecen en < min_df documentos
    """
    if stopwords_set is None:
        stopwords_set = set()

    # CountVectorizer respeta '_' si usamos token_pattern amplio
    vect = CountVectorizer(
        ngram_range=ngram_range,
        lowercase=False,
        token_pattern=r'(?u)\b\w+\b',
        min_df=min_df,
        stop_words=list(stopwords_set) if stopwords_set else None
    )
    X = vect.fit_transform(texts)
    freqs = np.asarray(X.sum(axis=0)).ravel()
    terms = np.array(vect.get_feature_names_out())
    order = np.argsort(freqs)[::-1][:top_k]
    out = pd.DataFrame({
        "ngram": terms[order],
        "freq": freqs[order],
        "df": np.asarray((X > 0).sum(axis=0)).ravel()[order]
    })
    return out

# --- 1A) N-gramas de TEMAS (contenido) ---
# Usamos text_topics (lemas + MWE de servicio, sin marcas) y quitamos stopwords generales+dominio
texts_topics = df["text_topics"].fillna("").astype(str).tolist()
top_bi_topics  = top_ngrams(texts_topics, (2,2), top_k=50, stopwords_set=sw_temas, min_df=2)
top_tri_topics = top_ngrams(texts_topics, (3,3), top_k=50, stopwords_set=sw_temas, min_df=2)

print("\nTop 20 BIGRAMAS (TEMAS):")
print(top_bi_topics.head(20).to_string(index=False))

print("\nTop 20 TRIGRAMAS (TEMAS):")
print(top_tri_topics.head(20).to_string(index=False))


c:\Users\santi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['Compañía', 'Fiduciaria', 'La', 'Positiva', 'Previsora', 'Seguros', 'axa', 'bolivar', 'colpatria', 'hdi', 'liberty'] not in stop_words.
  warnings.warn(



Top 20 BIGRAMAS (TEMAS):
                ngram  freq   df
              https t  7579 3275
                 t co  7579 3275
       seguro bolivar  2339 1747
           grupo argo  2328 1477
      junta directivo   752  523
      argo grupo_sura   701  529
         cemento argo   642  437
        axa colpatria   628  463
     grupo_sura grupo   536  415
      mensaje directo   504  251
    ricardo jaramillo   498  332
      mensaje privado   478  239
      compania seguro   469  361
 fiduciaria previsora   450  405
presidente grupo_sura   430  313
    acción grupo_sura   390  301
          previsora s   368  324
      seguro colombia   356  279
         grupo nutrés   354  296
       liberty seguro   333  264

Top 20 TRIGRAMAS (TEMAS):
                                    ngram  freq   df
                               https t co  7579 3275
                    grupo argo grupo_sura   607  463
                    grupo_sura grupo argo   423  315
   davidracero positivacol colombiacompra 

In [16]:
# ============================
# PALABRAS POCO FRECUENTES + FORMAS DE MENCIÓN
# ============================
import re
from collections import Counter
import numpy as np
import pandas as pd

# --- 2A) Rare terms (sin stopwords generales) ---
# Usamos tokens "lemma" para tener formas comparables, PERO no removemos marcas
# (porque queremos ver variantes). Sí quitamos stopwords generales.
def rare_terms_from_tokens(list_of_token_lists, stopwords_set, min_len=3, max_freq=3):
    """
    Retorna un DataFrame con términos cuya frecuencia total <= max_freq,
    excluyendo stopwords y tokens muy cortos/no alfanuméricos.
    """
    counter = Counter()
    for toks in list_of_token_lists:
        for t in toks:
            if t and len(t) >= min_len and re.search(r"[a-z0-9áéíóúñ_]", t):
                if t not in stopwords_set:
                    counter[t] += 1
    items = [(t, f) for t, f in counter.items() if f <= max_freq]
    out = pd.DataFrame(items, columns=["term", "freq"]).sort_values(["freq","term"])
    return out

rare_terms = rare_terms_from_tokens(
    df["tokens_lemma"].apply(lambda x: x if isinstance(x, list) else []).tolist(),
    stopwords_set=sw_es,
    min_len=3,
    max_freq=3,
)


print("\nRARE TERMS (<=3): muestra de 30")
print(rare_terms.head(30).to_string(index=False))
print(f"Total rare terms (<=3): {len(rare_terms)}")

# Puedes filtrar candidates que "suenen a marca" (porvenir, sura, bolivar, mapfre, allianz...)
looks_like_brand = rare_terms[rare_terms["term"].str.contains(
    r"(sura|bolivar|bolívar|mapfre|allianz|colpatria|zurich|liberty|hdi|porvenir)",
    case=False, regex=True
)]
print("\nRARE TERMS que parecen marcas/variantes (muestra 30):")
print(looks_like_brand.head(30).to_string(index=False))


# --- 2B) N-gramas que incluyen una marca (formas de mención) ---
# Tomamos text_blocked (con MWE unidas por '_') para capturar cosas como 'grupo_sura'
texts_blocked = df["text_blocked"].fillna("").astype(str).tolist()

# Construimos un set plano de alias (ya está en minúsculas en tu pipeline)
brand_alias_flat = set()
for brand, aliases in BRANDS.items():
    brand_alias_flat.add(brand)
    for a in aliases:
        brand_alias_flat.add(a.lower())

# Un pequeño vectorizador SIN stopwords para no perder marcas
def ngram_counts_including_brands(texts, ngram_range=(2,2), min_df=1):
    vect = CountVectorizer(
        ngram_range=ngram_range, lowercase=False,
        token_pattern=r'(?u)\b\w+\b', min_df=min_df
    )
    X = vect.fit_transform(texts)
    terms = np.array(vect.get_feature_names_out())
    freqs = np.asarray(X.sum(axis=0)).ravel()
    dfs   = np.asarray((X > 0).sum(axis=0)).ravel()
    df_terms = pd.DataFrame({"ngram": terms, "freq": freqs, "df": dfs})
    # filtra los n-gramas que contienen algún alias/marca
    mask = df_terms["ngram"].apply(
        lambda s: any(b in s for b in brand_alias_flat)
    )
    return df_terms[mask].sort_values("freq", ascending=False)

brand_bigrams  = ngram_counts_including_brands(texts_blocked, (2,2), min_df=1)
brand_trigrams = ngram_counts_including_brands(texts_blocked, (3,3), min_df=1)

print("\nFORMAS DE MENCIÓN (BIGRAMAS con marca) - Top 30:")
print(brand_bigrams.head(30).to_string(index=False))

print("\nFORMAS DE MENCIÓN (TRIGRAMAS con marca) - Top 30:")
print(brand_trigrams.head(30).to_string(index=False))



RARE TERMS (<=3): muestra de 30
                   term  freq
             "bienestar     1
                    "el     1
         "investigacion     1
                   )por     1
                 +0,16%     1
                  +0,2%     1
                 +0,51%     1
                  +0,6%     1
                 +0,64%     1
                  +0,7%     1
                 +0,84%     1
                  +0,86     1
                  +0,89     1
                 +0,98%     1
       +0.17%118,245.45     1
        +0.32%12,540.00     1
     +0.33%21,178.58btc     1
                 +0.60%     1
   +0.70%17,380.00dolar     1
+0.86%47,060.00grupoarg     1
                 +1,04%     1
                 +1,26%     1
                 +1,42%     1
                 +1,59%     1
                  +1,69     1
                 +1,71%     1
                 +1,96%     1
                 +1.03%     1
        +1.22%18,137.85     1
      +1.26%4,159.67bvc     1
Total rare terms (<=3): 19774

RARE T

C:\Users\santi\AppData\Local\Temp\ipykernel_13576\3704906943.py:40: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  looks_like_brand = rare_terms[rare_terms["term"].str.contains(



FORMAS DE MENCIÓN (BIGRAMAS con marca) - Top 30:
                     ngram  freq   df
              la previsora  2582 1776
           seguros bolivar  2342 1750
             de grupo_sura  1900 1371
              seguros sura  1115  729
            del grupo_sura   861  622
              grupo_sura y   835  655
             axa colpatria   657  483
             el grupo_sura   648  491
              y grupo_sura   618  460
         previsora seguros   541  296
       segurosbolivar hola   430  215
               previsora s   406  353
             en grupo_sura   347  266
           liberty seguros   333  264
             grupo bolivar   314  263
                    sura y   303  241
positivacol colombiacompra   300  150
   davidracero positivacol   298  149
                   de sura   293  236
          allianz colombia   274  223
 davivienda segurosbolivar   248  127
                  eps sura   247  159
            con grupo_sura   246  185
             grupo_sura en   245  184


Primeros Ajustes:


In [20]:
# --- 1A) Ampliar MWE (servicio) y entidades/noticioso/CM ---
MWE_SERVICE += [
    "pago de siniestro","pago de la póliza","tiempo de respuesta","atención al cliente",
    "servicio al cliente","línea de atención","red médica","autorización de servicio",
    "incremento de prima","renovación de póliza","radicado de pqrs","letra pequeña"
]
MWE_ENTITIES += [
    "grupo argos","grupo sura","grupo empresarial antioqueño","suramericana s a",
    "inversiones suramericana","fundacion mapfre","allianz colombia","seguros del estado"
]

# --- 1B) Alias/typos para marcas (gazetteer) ---
BRANDS["seguros bolivar"] |= {
    "seguros bolívar","segurosbolivar","#segurosbolivar","#segurosbolívar",
    "grupo bolívar","grupo bolivar","grupobolivar","agrobolivar","bolivar seguros"
}
BRANDS["sura"] |= {
    "grupo sura","gruposura","grup_sura","grupsura","suramericana","eps sura",
    "inversiones suramericana","suramericana s a","@suramericana","segurossura"
}
BRANDS["mapfre"] |= {"fundacion mapfre","mapfre col","mapfre_col","2024mapfre"}
BRANDS["allianz"] |= {"allianz colombia","allianz.co","allianzcol","allianzcolombio"}
BRANDS["axa colpatria"] |= {"axacolpatria","colpatria seguros","axacolpatriar"}
BRANDS["liberty seguros"] |= {"liberty seguros","legal-liberty"}
BRANDS["seguros del estado"] |= {"seguros del estado"}
BRANDS["zurich"] |= {"zurich colombia"}
BRANDS["hdi seguros"] |= {"hdi seguros"}
BRANDS["porvenir"] |= {"porvenir","fondo de pensiones obligatorias porvenir"}

# --- 1C) Dominios por sufijo ---
DOMAIN_SUFFIX2BRAND = {
    "segurosbolivar.com":"seguros bolivar","grupobolivar.com":"seguros bolivar",
    "segurossura.com":"sura","sura.com":"sura","epssura.com":"sura",
    "axacolpatria.co":"axa colpatria","allianz.co":"allianz","mapfre.com":"mapfre",
    "fundacionmapfre.org":"mapfre","libertycolombia.com":"liberty seguros",
    "zurich.com.co":"zurich","hdi.com.co":"hdi seguros","segurosdelestado.com":"seguros del estado",
    "porvenir.com.co":"porvenir"
}

# --- 1D) Stopwords de dominio (solo para TEMAS) ---
DOMAIN_STOPWORDS |= {
    "seguro","seguros","aseguradora","aseguradoras","poliza","póliza","cliente","clientes",
    "grupo","fundacion","fundación","colombia","colombiano","colombianos",
    *set(BRANDS.keys())
}

# --- 1E) Normalizaciones canónicas antes de tokenizar ---
CANON_REPL = [
    (r"\bjunta directivo\b","junta directiva"),
    (r"\bcemento argo\b","cementos argos"),
    (r"\bprevisora s\b","previsora s.a."),
    (r"\bseguro bolivar\b","seguros bolivar"),
    (r"\bseguro surar\b","seguros sura"),
    (r"\bcompan[ií]a seguro\b","compañía de seguros"),
    (r"\bgrupo nutr[ée]s?\b","grupo nutresa"),
]
def canonicalize(text: str) -> str:
    t = text
    for pat, rep in CANON_REPL:
        t = re.sub(pat, rep, t, flags=re.IGNORECASE)
    return t

# --- 1F) Flags para filtrar corporativo y respuestas de CM ---
CORP_NEWS_PATTERNS = {
    "grupo_sura","grupo argos","argos grupo","junta directiv","presidente",
    "grupo empresarial antioque","acciones","accionistas",
    "inversiones suramericana","suramericana s a","edificio seguros bolivar"
}
CM_TEMPLATES = {
    "mensaje directo","mensaje privado","favor escrib","invitamos contactar",
    "pagina oficial","facebook","instagram","contactarno pagina","medio mensaje directo"
}
def is_corporate_news(t: str) -> bool:
    return any(pat in ascii_fold(t) for pat in CORP_NEWS_PATTERNS)
def is_cm_template(t: str) -> bool:
    return any(pat in ascii_fold(t) for pat in CM_TEMPLATES)

# --- 1G) extract_brands actualizado (texto + dominio por sufijo) ---
def extract_brands(text_norm: str, domain: str = "") -> list:
    found = set()
    t = " " + ascii_fold(text_norm) + " "
    for brand, aliases in BRANDS.items():
        if any((" " + ascii_fold(a) + " ") in t for a in (aliases | {brand})):
            found.add(brand)
    d = (domain or "").lower().strip()
    d = d.replace("https://","").replace("http://","").split("/")[0]
    d = d[4:] if d.startswith("www.") else d
    for suffix, brand in DOMAIN_SUFFIX2BRAND.items():
        if d.endswith(suffix):
            found.add(brand)
    return sorted(found)


In [21]:
# Re-aplica limpieza canónica
df["text_clean_base"] = df["text_clean_base"].map(canonicalize)

# Re-bloquea MWE (entidades primero, luego servicio)
df["text_mwe_entities"] = df["text_clean_base"].map(lambda t: block_mwes(t, MWE_ENTITIES))
df["text_blocked"] = df["text_mwe_entities"].map(lambda t: block_mwes(t, MWE_SERVICE))

# Recalcula marcas
df["brand_list"] = df.apply(lambda r: extract_brands(r["text_blocked"], r.get(COL_DOMAIN, "")), axis=1)
df["brand_primary"] = df.apply(lambda r: pick_primary_brand(r["brand_list"], r["text_blocked"]), axis=1)

# Flags corporativo/CM
df["flag_corp_news"] = df["text_clean_base"].map(is_corporate_news)
df["flag_cm_reply"]  = df["text_clean_base"].map(is_cm_template)

# Tokens y textos para TEMAS (lema + sin marcas)
df["tokens_lemma"] = df["text_blocked"].map(tokenize_lemma)
df["tokens_topics"] = df["tokens_lemma"].map(lambda xs: [t for t in xs if t not in DOMAIN_STOPWORDS])
df["text_topics"]   = df["tokens_topics"].map(lambda xs: " ".join(xs))

# Subconjunto tema-centrado (sug.) para sacar n-gramas “limpios”
df_topics = df[~df["flag_corp_news"] & ~df["flag_cm_reply"]].copy()
len(df), len(df_topics)


(15469, 10684)

In [22]:
# --- TOP N-GRAMAS (TEMAS) sobre df_topics ---
texts_topics = df_topics["text_topics"].fillna("").astype(str).tolist()

top_bi_topics  = top_ngrams(texts_topics, (2,2), top_k=50, stopwords_set=sw_temas, min_df=2)
top_tri_topics = top_ngrams(texts_topics, (3,3), top_k=50, stopwords_set=sw_temas, min_df=2)

print("\nTop 20 BIGRAMAS (TEMAS, filtrado corporativo/CM):")
print(top_bi_topics.head(20).to_string(index=False))
print("\nTop 20 TRIGRAMAS (TEMAS, filtrado corporativo/CM):")
print(top_tri_topics.head(20).to_string(index=False))

# --- RARE TERMS (para detectar alias) en df (global) ---
brand_regex = r"(sura|bol[ií]var|mapfre|allianz|colpatria|zurich|liberty|hdi|porvenir|suramericana|argos)"
token_lists = df["tokens_lemma"].apply(
    lambda x: x if isinstance(x, list)
    else ([] if pd.isna(x) else (x.split() if isinstance(x, str) else []))
)
rare_terms = rare_terms_from_tokens(
    token_lists.tolist(),
    stopwords_set=sw_es,
    min_len=3,
    max_freq=3
)
looks_like_brand = rare_terms[rare_terms["term"].str.contains(brand_regex, case=False, regex=True, na=False)]
print("\nRARE TERMS que parecen marcas/variantes (muestra 30):")
print(looks_like_brand.head(30).to_string(index=False))

# --- FORMAS DE MENCIÓN (n-gramas con marca) sobre df (global) ---
texts_blocked = df["text_blocked"].fillna("").astype(str).tolist()
brand_bigrams  = ngram_counts_including_brands(texts_blocked, (2,2), min_df=1)
brand_trigrams = ngram_counts_including_brands(texts_blocked, (3,3), min_df=1)

print("\nFORMAS DE MENCIÓN (BIGRAMAS con marca) - Top 30:")
print(brand_bigrams.head(30).to_string(index=False))
print("\nFORMAS DE MENCIÓN (TRIGRAMAS con marca) - Top 30:")
print(brand_trigrams.head(30).to_string(index=False))


c:\Users\santi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['Compañía', 'Fiduciaria', 'La', 'Positiva', 'Previsora', 'Seguros', 'axa', 'bolivar', 'colpatria', 'hdi', 'liberty'] not in stop_words.
  warnings.warn(



Top 20 BIGRAMAS (TEMAS, filtrado corporativo/CM):
                        ngram  freq   df
                         t co  5835 2462
                      https t  5835 2462
   positivacol colombiacompra   300  150
      davidracero positivacol   293  146
colombiacompra wradiocolombia   290  145
                sistema salud   222  151
        dcoronell davidracero   210  105
                    nuevo eps   199  106
                       si ser   191  116
                      ser mas   189  125
            primero trimestre   186  119
               integral salud   180   87
       aseguramiento integral   180   87
             primero semestre   175  118
                utilidad neta   171  111
           patinaje velocidad   167   84
   daviviendir segurosbolivar   165   92
           davivienda bolivar   164  140
                    haber ser   159  116
               riesgo laboral   144   76

Top 20 TRIGRAMAS (TEMAS, filtrado corporativo/CM):
                                    

C:\Users\santi\AppData\Local\Temp\ipykernel_13576\1020284944.py:24: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  looks_like_brand = rare_terms[rare_terms["term"].str.contains(brand_regex, case=False, regex=True, na=False)]



RARE TERMS que parecen marcas/variantes (muestra 30):
                                                                         term  freq
                                                               -axa_colpatria     1
                                                                   -colpatria     1
                                                                 -grupo_argos     1
                                                             0,00%.grupo_sura     1
                                                             1.07%.grupo_sura     1
                                                             2,45%.grupo_sura     1
                                                                   2024mapfre     1
                                                                  agrobolivar     1
                                                                   allianz.co     1
                                                               allianz.sandra     1
                     

In [23]:
# --- ENTIDADES (bloqueo como unidad, no para TEMAS) ---
MWE_ENTITIES += [
    "fiduciaria previsora", "previsora s.a.", "axa colpatria"
]

# --- BRANDS (alias/typos detectados) ---
BRANDS.setdefault("fiduciaria previsora", set())
BRANDS["fiduciaria previsora"] |= {"fiduciaria previsora", "previsora s", "previsora s.a.", "previsora"}

BRANDS["axa colpatria"] |= {"axacolpatria", "-colpatria"}  # aparece en rare terms

DOMAIN_STOPWORDS |= {
    "fiduciaria", "previsora", "previsora s.a.", "axa", "colpatria", "axa colpatria"
}
# Amplía los patrones de CM
CM_TEMPLATES |= {
    "dejar link", "link dej", "aqui dejar link",
    "atenderte manera", "atenderte manera directo",
    "herramienta atenderte", "solventar caso", "poder solventar caso",
    "medio mensaje directo", "favor escribano mensaje", "mensaje directo atento"
}

MWE_ENTITIES += [
    "dejar link", "atenderte manera directo", "herramienta atenderte manera",
    "solventar caso manera"
]

CORP_NEWS_PATTERNS |= {
    "utilidad neta", "primero trimestre", "primer trimestre",
    "primero semestre", "primer semestre", "value and risk",
    "patinaje velocidad", "davivienda bolivar", "nuevo modelo salud",
    "aseguramiento integral salud", "sistema salud", "integral salud",
    "seguridad salud trabajo", "fondo nacional", "gestion riesgo", "riesgo laboral"
}

STOPWORDS_EXTRA = {
    "ser","haber","poder","mas","si","nuevo","empresa","nacional","gestión","riesgo","integral"
}
DOMAIN_STOPWORDS |= STOPWORDS_EXTRA

CANON_REPL += [
    (r"\bprevisora s\b","previsora s.a."),
    (r"\bvalue and risk\b","value and risk"),  # si decides filtrar, basta con el patrón en CORP_NEWS_PATTERNS
    (r"\bprimero semestre\b","primer semestre"),
    (r"\bprimero trimestre\b","primer trimestre"),
]

NUMPCT_TOKEN_RE = re.compile(r"(?=.*\d)(?=.*[%\.,])")  # contiene dígitos y %/./, mezclados

def remove_num_pct_tokens(tokens):
    return [t for t in tokens if not NUMPCT_TOKEN_RE.search(t)]

# aplica sólo a TEMAS:
df["tokens_topics"] = df["tokens_lemma"].map(lambda xs: remove_num_pct_tokens(
    [t for t in xs if t not in DOMAIN_STOPWORDS]
))
df["text_topics"] = df["tokens_topics"].map(lambda xs: " ".join(xs))

NUMPCT_TOKEN_RE = re.compile(r"(?=.*\d)(?=.*[%\.,])")  # contiene dígitos y %/./, mezclados

def remove_num_pct_tokens(tokens):
    return [t for t in tokens if not NUMPCT_TOKEN_RE.search(t)]

# aplica sólo a TEMAS:
df["tokens_topics"] = df["tokens_lemma"].map(lambda xs: remove_num_pct_tokens(
    [t for t in xs if t not in DOMAIN_STOPWORDS]
))
df["text_topics"] = df["tokens_topics"].map(lambda xs: " ".join(xs))

# Reaplica canonicalize
df["text_clean_base"] = df["text_clean_base"].map(canonicalize)

# Rebloquea MWE
df["text_mwe_entities"] = df["text_clean_base"].map(lambda t: block_mwes(t, MWE_ENTITIES))
df["text_blocked"] = df["text_mwe_entities"].map(lambda t: block_mwes(t, MWE_SERVICE))

# Recalcula marcas + flags
df["brand_list"] = df.apply(lambda r: extract_brands(r["text_blocked"], r.get(COL_DOMAIN, "")), axis=1)
df["brand_primary"] = df.apply(lambda r: pick_primary_brand(r["brand_list"], r["text_blocked"]), axis=1)
df["flag_corp_news"] = df["text_clean_base"].map(is_corporate_news)
df["flag_cm_reply"]  = df["text_clean_base"].map(is_cm_template)

# Tokens/temas
df["tokens_lemma"] = df["text_blocked"].map(tokenize_lemma)
df["tokens_topics"] = df["tokens_lemma"].map(lambda xs: remove_num_pct_tokens([t for t in xs if t not in DOMAIN_STOPWORDS]))
df["text_topics"]   = df["tokens_topics"].map(lambda xs: " ".join(xs))

# Subconjunto tema-centrado
df_topics = df[~df["flag_corp_news"] & ~df["flag_cm_reply"]].copy()


In [31]:
# --- TOP N-GRAMAS (TEMAS) sobre df_topics ---
texts_topics = df_topics["text_topics"].fillna("").astype(str).tolist()

top_bi_topics  = top_ngrams(texts_topics, (2,2), top_k=50, stopwords_set=sw_temas, min_df=2)
top_tri_topics = top_ngrams(texts_topics, (3,3), top_k=50, stopwords_set=sw_temas, min_df=2)

print("\nTop 20 BIGRAMAS (TEMAS, filtrado corporativo/CM):")
print(top_bi_topics.head(20).to_string(index=False))
print("\nTop 20 TRIGRAMAS (TEMAS, filtrado corporativo/CM):")
print(top_tri_topics.head(20).to_string(index=False))

# --- RARE TERMS (para detectar alias) en df (global) ---
brand_regex = r"(sura|bol[ií]var|mapfre|allianz|colpatria|zurich|liberty|hdi|porvenir|suramericana|argos)"
token_lists = df["tokens_lemma"].apply(
    lambda x: x if isinstance(x, list)
    else ([] if pd.isna(x) else (x.split() if isinstance(x, str) else []))
)
rare_terms = rare_terms_from_tokens(
    token_lists.tolist(),
    stopwords_set=sw_es,
    min_len=3,
    max_freq=3
)
looks_like_brand = rare_terms[rare_terms["term"].str.contains(brand_regex, case=False, regex=True, na=False)]
print("\nRARE TERMS que parecen marcas/variantes (muestra 30):")
print(looks_like_brand.head(30).to_string(index=False))

# --- FORMAS DE MENCIÓN (n-gramas con marca) sobre df (global) ---
texts_blocked = df["text_blocked"].fillna("").astype(str).tolist()
brand_bigrams  = ngram_counts_including_brands(texts_blocked, (2,2), min_df=1)
brand_trigrams = ngram_counts_including_brands(texts_blocked, (3,3), min_df=1)

print("\nFORMAS DE MENCIÓN (BIGRAMAS con marca) - Top 30:")
print(brand_bigrams.head(30).to_string(index=False))
print("\nFORMAS DE MENCIÓN (TRIGRAMAS con marca) - Top 30:")
print(brand_trigrams.head(30).to_string(index=False))

c:\Users\santi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['Compañía', 'Fiduciaria', 'La', 'Positiva', 'Previsora', 'Seguros', 'axa', 'bolivar', 'colpatria', 'hdi', 'liberty'] not in stop_words.
  warnings.warn(



Top 20 BIGRAMAS (TEMAS, filtrado corporativo/CM):
                        ngram  freq  df
                      https t   971 469
                         t co   971 469
   positivacol colombiacompra   300 150
      davidracero positivacol   293 146
colombiacompra wradiocolombia   290 145
        dcoronell davidracero   210 105
          aseguramiento salud   208 100
                sistema salud   208 140
           patinaje velocidad   167  84
   daviviendir segurosbolivar   165  92
           davivienda bolivar   162 138
                salud bolivar   131  81
                 modelo salud   127  94
       supersalud positivacol   125  65
     daviplata segurosbolivar   125  65
                 salud mental   122  66
    segurosbolivar davivienda   122  62
         modelo aseguramiento   120  48
                 millón pesos   117  97
             senor supersalud   116  59

Top 20 TRIGRAMAS (TEMAS, filtrado corporativo/CM):
                                    ngram  freq  df
     

C:\Users\santi\AppData\Local\Temp\ipykernel_13576\350065347.py:24: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  looks_like_brand = rare_terms[rare_terms["term"].str.contains(brand_regex, case=False, regex=True, na=False)]



RARE TERMS que parecen marcas/variantes (muestra 30):
                                                                         term  freq
                                                               -axa_colpatria     1
                                                                   -colpatria     1
                                                                 -grupo_argos     1
                                                             0,00%.grupo_sura     1
                                                             1.07%.grupo_sura     1
                                                             2,45%.grupo_sura     1
                                                                   2024mapfre     1
                                                                  agrobolivar     1
                                                                   allianz.co     1
                                                               allianz.sandra     1
                     

In [37]:
top_bi_topics


,ngram,freq,df
0,https t,971,469
1,t co,971,469
2,positivacol colombiacompra,300,150
3,davidracero positivacol,293,146
4,colombiacompra wradiocolombia,290,145
5,dcoronell davidracero,210,105
6,aseguramiento salud,208,100
7,sistema salud,208,140
8,patinaje velocidad,167,84
9,daviviendir segurosbolivar,165,92


# Exploración de Kmeans
- representar con TF-IDF (1–2),
- generar temas iniciales sin etiquetas (clustering),
- bautizarlos con nombres tentativos a partir de sus términos top,
- cruzarlos con marcas para ver distribución,
- exportar un dataset listo para revisión/etiquetado fino.

In [40]:
# ============================================================
# Filtrado adicional: excluir noticias corporativas y respuestas CM
# ============================================================
import re

# --- FINANZAS / CORPORATIVO ---
# Evita que noticias sobre utilidades o reportes financieros sesguen los clusters
CORP_NEWS_PATTERNS |= {
    "billon", "utilidad", "utilidades", "colcap", "accion", "acciones", "preferencial",
    "ranking", "trimestre", "semestre", "msci", "bolsa", "capitalizacion", "ingresos",
    "resultado oficial", "valoracion", "dividendo", "emision", "empresa mas grande"
}

# --- RESPUESTAS DE COMMUNITY MANAGER ---
# Detecta mensajes genéricos sin contenido (para no agruparlos como “temas”)
CM_TEMPLATES |= {
    "por interno", "escribenos por interno", "escríbenos por interno",
    "escribenos al dm", "escríbenos al dm", "canales oficiales", "link en",
    "aqui te dejamos el link", "dejanos tus datos", "dejar tus datos",
    "mensaje directo", "favor escribanos", "contactanos por interno"
}

# --- FUNCIONES DE MARCADO ---
def is_corporate_news(text):
    t = text.lower()
    return any(pat in t for pat in CORP_NEWS_PATTERNS)

def is_cm_template(text):
    t = text.lower()
    return any(pat in t for pat in CM_TEMPLATES)

# --- Recalcular banderas (flags) y df_topics ---
df["flag_corp_news"] = df["text_clean_base"].map(is_corporate_news)
df["flag_cm_reply"]  = df["text_clean_base"].map(is_cm_template)

# Excluir estos tweets para concentrarse en menciones con contenido real
df_topics = df[~df["flag_corp_news"] & ~df["flag_cm_reply"]].copy()

print(f"Corpus final para KMeans: {len(df_topics)} tweets (limpio de noticias y CM)")


Corpus final para KMeans: 9406 tweets (limpio de noticias y CM)


In [41]:
# ============================================
# Representación TF-IDF (1–2) + KMeans (temas)
#  - Limpieza extra: exclusión de n-gramas/términos ruidosos
# ============================================
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.pairwise import cosine_distances

# -------- 0) Entradas / corpus tema-centrado --------
# Usamos df_topics, creado antes (sin corporativo/CM). Si no existe, cae a df.
if 'df_topics' not in globals():
    df_topics = df[~df["flag_corp_news"] & ~df["flag_cm_reply"]].copy()

texts = df_topics["text_topics"].fillna("").astype(str).tolist()
doc_index = df_topics.index.to_list()  # para mapear de vuelta

# -------- 1) Lista de exclusión (n-gramas y términos ruidosos) --------
EXCLUDE_TERMS = [
    # --- URLs y tokens genéricos ---
    "https", "t", "co", "https t", "t co", "24 7",

    # --- Marcas y entidades ---
    "segurosbolivar", "davivienda", "daviviendir", "bolivar",
    "grupo_sura", "sura", "axa_colpatria", "axacolpatria",
    "mapfre_co", "liberty", "positivacol", "positiva",
    "previsora", "previsoro", "previsora_s", "previsora s.a.",
    "hdi", "zurich", "colpatria", "fomag", "fasecolda",
    "sfcsupervisor", "sicsuper", "supersalud", "superintendencia financiero",
    "defensoriacol", "daviplata", "banco davivienda",

    # --- N-gramas compuestos de marcas ---
    "daviviendir segurosbolivar", "segurosbolivar davivienda",
    "segurosbolivar daviplata", "daviplata segurosbolivar",
    "davivienda bolivar", "bolivar davivienda",
    "salud bolivar", "daviviendir bolivar", "compania bolivar",
    "positivo compania", "positivacol colombiacompra",
    "supersalud positivacol", "senor supersalud",
    "cemento argo", "preferencial grupo_sura",
    "app surar", "trav app",

    # --- Institucional / financiero ---
    "sector asegurador", "msci colcap", "millón pesos", "ano pasado",
    "resultado oficial", "2024 patinaje", "modelo salud",
    "modelo aseguramiento", "modelo sostenible",
    "sostenible aseguramiento", "aseguramiento salud",
    "seguridad salud", "sistema salud", "reforma salud",
    "salud magisterio", "salud mental",

    # --- Expresiones genéricas vacías de contenido ---
    "hacer parte", "llevar cabo", "caso manera",
    "aqui dejar", "manera rapido", "dejar link"
]

# Preparamos regex para n-gramas (multi-palabra) y términos de 1 palabra
EXCLUDE_MULTI = sorted([t for t in EXCLUDE_TERMS if " " in t], key=len, reverse=True)
EXCLUDE_UNI   = {t for t in EXCLUDE_TERMS if " " not in t}

def strip_excluded_ngrams(text: str) -> str:
    """Elimina n-gramas multi-palabra usando límites de palabra; colapsa espacios."""
    if not text:
        return text
    t = f" {text} "
    for ng in EXCLUDE_MULTI:
        # \b para límites, espacios convertidos a \s+ para tolerar múltiples espacios
        pat = re.compile(rf'(?i)(?<!\w){re.escape(ng).replace(r"\ ", r"\s+")}(?!\w)')
        t = pat.sub(" ", t)
    # limpia espacios
    t = re.sub(r"\s+", " ", t).strip()
    return t

# Aplicamos la limpieza de n-gramas ANTES de vectorizar
texts_clean = [strip_excluded_ngrams(x) for x in texts]

# -------- 2) TF-IDF (unigramas + bigramas) --------------
# Stopwords: español NLTK + personalizadas + unigrams de exclusión
from nltk.corpus import stopwords
import nltk
try:
    _ = stopwords.words('spanish')
except LookupError:
    nltk.download('stopwords', quiet=True)

stop_es = set(stopwords.words('spanish')) | {"https", "t", "co", "él", "la", "los", "las"}
# añadimos los UNIGRAMAS de la lista de exclusión a las stopwords
stop_es = stop_es | EXCLUDE_UNI

tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=5,
    max_df=0.6,
    max_features=50000,
    lowercase=False,
    stop_words=sorted(list(stop_es)),  # sklearn requiere list o 'english'
    token_pattern=r'(?u)\b\w+\b'
)

X = tfidf.fit_transform(texts_clean)
terms = np.array(tfidf.get_feature_names_out())

print(f"TF-IDF: docs={X.shape[0]}, vocab={X.shape[1]}")

# -------- 3) KMeans (elige k) ---------------------------
# Sugerencia: 6–8 temas iniciales para este dominio
K = 7
km = MiniBatchKMeans(n_clusters=K, random_state=42, batch_size=2048, n_init='auto')
labels = km.fit_predict(X)

df_topics["topic_km"] = labels

# Reporte de tamaños por cluster
sizes = np.bincount(labels, minlength=K)
print("Tamaños por cluster:", sizes.tolist())

# -------- 4) Top términos por cluster -------------------
def top_terms_per_cluster(X, labels, terms, topn=15):
    centroids = km.cluster_centers_
    tops = {}
    for k in range(centroids.shape[0]):
        idx = np.argsort(centroids[k])[::-1][:topn]
        tops[k] = list(terms[idx])
    return tops

cluster_terms = top_terms_per_cluster(X, labels, terms, topn=15)

print("\nTop términos por cluster (primeras 15 features):")
for k, feats in cluster_terms.items():
    print(f"\nCluster {k}: {', '.join(feats)}")

# -------- 5) Muestras representativas por cluster -------
def representative_docs(X, labels, n=5):
    reps = {}
    centroids = km.cluster_centers_
    dists = cosine_distances(X, centroids)
    for k in range(centroids.shape[0]):
        idx_k = np.where(labels == k)[0]
        if len(idx_k) == 0:
            reps[k] = []
            continue
        order = np.argsort(dists[idx_k, k])[:n]
        reps[k] = idx_k[order]
    return reps

rep = representative_docs(X, labels, n=5)

print("\nEjemplos representativos por cluster (índices de df_topics):")
for k, idxs in rep.items():
    print(f"\nCluster {k} -> {len(idxs)} ejemplos")
    for i in idxs:
        row = df_topics.iloc[i]
        txt_preview = str(row.get('text_raw', row.get('text_clean_base', '')))[:140].replace('\n',' ')
        print(f"  · id={row.get('id', '')} | brand={row.get('brand_primary','')} | text={txt_preview}...")

# -------- 6) Tentativa de nombres de tema ----------------
# Heurística inicial (ajústala leyendo 'cluster_terms' y muestras):
topic_names = {
    0: "Pagos/Siniestros",
    1: "Atención/Trámite",
    2: "Cobertura/Condiciones",
    3: "Red/Autorizaciones",
    4: "Costos/Primas",
    5: "Salud/EPS/Regulación",
    6: "Otros/Mixto"
}
df_topics["topic_name"] = df_topics["topic_km"].map(topic_names).fillna("Tema_sin_nombre")

# -------- 7) Cruce con marcas (distribución) -------------
df_out = df.copy()
df_out.loc[df_topics.index, "topic_km"] = df_topics["topic_km"]
df_out.loc[df_topics.index, "topic_name"] = df_topics["topic_name"]

tab_global = (
    df_topics["topic_name"].value_counts()
    .rename_axis("topic_name")
    .reset_index(name="n_docs")
)
tab_brand  = (
    df_out.dropna(subset=["topic_name"])
         .groupby(["brand_primary","topic_name"])
         .size().reset_index(name="n_docs")
         .sort_values(["brand_primary","n_docs"], ascending=[True, False])
)

print("\nDistribución global de temas (KMeans):")
print(tab_global.to_string(index=False))

print("\nDistribución por marca y tema (primeras filas):")
print(tab_brand.head(20).to_string(index=False))

# -------- 8) Export para revisión/etiquetado -------------
cols_export = [
    "id", "Date", "brand_primary", "brand_list",
    "text_raw", "text_clean_base", "text_topics",
    "topic_km", "topic_name"
]
export_topics_path = "corpus_con_temas_kmeans2.csv"
df_out[cols_export].to_csv(export_topics_path, index=False, encoding="utf-8")
print(f"\nExportado: {export_topics_path}")



TF-IDF: docs=9406, vocab=9341
Tamaños por cluster: [195, 1036, 5490, 546, 729, 500, 910]

Top términos por cluster (primeras 15 features):

Cluster 0: avenida, santander, calle, 15, calle 15, 2024, avenida santander, glorieta, edificio, cartagena, san, martin, pico, crespo, avenida san

Cluster 1: s, ungrd, salud, tener, compania, soat, carrotanqu, contrato, general, guajira, magisterio, entidad, seguros_del_estado, gestion, proceso

Cluster 2: ano, davidracero, wradiocolombia, millón, tener, dcoronell, banco, pais, davidracero wradiocolombia, asi, financiero, 2024, servicio, sector, social

Cluster 3: hacer, pagar, hora, tener, empleo, vacante, salario, 2, servicio, buscar, comercial, 2 hora, hacer 2, millón, trabajar

Cluster 4: llamar, decir, senor, hacer, seguir, servicio, nunca, dar, esperar, tener, cancelar, mes, cobrar, responder, dinero

Cluster 5: eps, salud, aseguramiento, estatal, aseguramiento salud, sistema, sistema salud, modelo, retiro, reforma, modelo aseguramiento, usu

In [42]:
# ============================================
# Nombramiento y guardado de clusters finales
# ============================================
import pandas as pd

# --- Asigna nombres interpretativos según análisis manual ---
topic_names_final = {
    0: "Movilidad/Pico y Placa Cartagena",
    1: "SOAT/Fraudes/Carrotanques/Entidades Públicas",
    2: "Política/Economía/Medios/Contexto Nacional",
    3: "Empleo/Salarios/Oportunidades Laborales",
    4: "Quejas/Reclamos/Descuentos No Autorizados",
    5: "Salud/EPS/Retiro y Reforma al Sistema",
    6: "Respuestas Automáticas/Community Management"
}

# Mapea los nombres en el DataFrame
df_topics["topic_name"] = df_topics["topic_km"].map(topic_names_final).fillna("Tema_sin_nombre")

# Fusiona con df completo (para conservar marcas y texto original)
df_out = df.copy()
df_out.loc[df_topics.index, "topic_km"] = df_topics["topic_km"]
df_out.loc[df_topics.index, "topic_name"] = df_topics["topic_name"]

# Tablas resumen
tab_global = (
    df_topics["topic_name"]
    .value_counts()
    .rename_axis("topic_name")
    .reset_index(name="n_docs")
)
tab_brand = (
    df_out.dropna(subset=["topic_name"])
    .groupby(["brand_primary", "topic_name"])
    .size()
    .reset_index(name="n_docs")
    .sort_values(["brand_primary", "n_docs"], ascending=[True, False])
)

# Mostrar resumen
print("\nDistribución global de temas finales:")
print(tab_global.to_string(index=False))

print("\nDistribución por marca y tema (primeras filas):")
print(tab_brand.head(20).to_string(index=False))

# --- Exportar resultados finales ---
cols_export = [
    "id", "Date", "brand_primary", "brand_list",
    "text_raw", "text_clean_base", "text_topics",
    "topic_km", "topic_name"
]
export_final_path = "corpus_con_temas_final.csv"
df_out[cols_export].to_csv(export_final_path, index=False, encoding="utf-8")

print(f"\n✅ Exportado correctamente: {export_final_path}")



Distribución global de temas finales:
                                  topic_name  n_docs
  Política/Economía/Medios/Contexto Nacional    5490
SOAT/Fraudes/Carrotanques/Entidades Públicas    1036
 Respuestas Automáticas/Community Management     910
   Quejas/Reclamos/Descuentos No Autorizados     729
     Empleo/Salarios/Oportunidades Laborales     546
       Salud/EPS/Retiro y Reforma al Sistema     500
            Movilidad/Pico y Placa Cartagena     195

Distribución por marca y tema (primeras filas):
               brand_primary                                   topic_name  n_docs
                               Política/Economía/Medios/Contexto Nacional    2393
                             SOAT/Fraudes/Carrotanques/Entidades Públicas     492
                              Respuestas Automáticas/Community Management     323
                                  Empleo/Salarios/Oportunidades Laborales     165
                                    Salud/EPS/Retiro y Reforma al Sistema     

# Clasificacion Supervisada

In [43]:
# ============================================
# CLASIFICADOR SUPERVISADO DE TEMAS (desde corpus_con_temas_final.csv)
# ============================================
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import nltk
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords

# -------- 1) Carga del corpus final -----------
df = pd.read_csv("corpus_con_temas_final.csv")
print("Corpus cargado:", df.shape)

# Verifica que existan las columnas esperadas
assert all(col in df.columns for col in ["text_topics", "topic_name"]), \
    "Faltan columnas necesarias ('text_topics', 'topic_name') en el CSV."

# Filtra filas válidas
df_sup = df.dropna(subset=["text_topics", "topic_name"]).copy()
print("Textos válidos para entrenamiento:", len(df_sup))

# -------- 2) Variables X / y -----------
X_text = df_sup["text_topics"].astype(str)
y = df_sup["topic_name"].astype(str)

# -------- 3) Split entrenamiento / prueba -----------
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X_text, y, df_sup.index, test_size=0.2, random_state=42, stratify=y
)

# -------- 4) Vectorización TF-IDF -----------
sw_es = set(stopwords.words('spanish'))
tfidf = TfidfVectorizer(
    ngram_range=(1,2),
    min_df=5,
    max_df=0.6,
    max_features=40000,
    lowercase=False,
    token_pattern=r'(?u)\b\w+\b',
    stop_words=list(sw_es)
)
Xtr = tfidf.fit_transform(X_train)
Xte = tfidf.transform(X_test)

# -------- 5) Modelo supervisado -----------
clf = LogisticRegression(
    max_iter=2000,
    class_weight="balanced",
    n_jobs=None,
    solver="lbfgs",
    multi_class="auto"
)
clf.fit(Xtr, y_train)

# -------- 6) Evaluación -----------
y_pred = clf.predict(Xte)
print("\nF1 macro (test):", round(f1_score(y_test, y_pred, average="macro"), 3))
print("\nReporte de clasificación (test):")
print(classification_report(y_test, y_pred, digits=3))

cm = confusion_matrix(y_test, y_pred, labels=sorted(y.unique()))
cm_df = pd.DataFrame(cm, index=[f"true:{c}" for c in sorted(y.unique())],
                        columns=[f"pred:{c}" for c in sorted(y.unique())])
print("\nMatriz de confusión:")
print(cm_df.to_string())

# -------- 7) Palabras más representativas por tema -----------
feature_names = np.array(tfidf.get_feature_names_out())
classes = clf.classes_
coefs = clf.coef_

def top_features_for_class(k=25):
    out = {}
    for i, c in enumerate(classes):
        idx = np.argsort(coefs[i])[::-1][:k]
        out[c] = feature_names[idx]
    return out

tops = top_features_for_class(25)
print("\nTop 25 términos por tema:")
for c, feats in tops.items():
    print(f"\n{c}: " + ", ".join(feats))

# -------- 8) Evaluación por marca -----------
test_df = df_sup.loc[idx_test].copy()
test_df["y_true"] = y_test.values
test_df["y_pred"] = y_pred

if "brand_primary" in df_sup.columns:
    brand_eval = (
        test_df.groupby("brand_primary")
               .apply(lambda g: pd.Series({
                   "n_test": len(g),
                   "f1_macro": f1_score(g["y_true"], g["y_pred"], average="macro")
               }))
               .sort_values("n_test", ascending=False)
    )
    print("\nF1 macro por marca (test) - top 15:")
    print(brand_eval.head(15).to_string())

# -------- 9) Export resultados -----------
pred_path = "pred_supervisado_temas_test.csv"
err_path  = "errores_supervisado_temas_test.csv"
test_df.to_csv(pred_path, index=False, encoding="utf-8")
test_df[test_df["y_true"] != test_df["y_pred"]].to_csv(err_path, index=False, encoding="utf-8")

print(f"\nExportados:\n- {pred_path}\n- {err_path}")


Corpus cargado: (15469, 9)
Textos válidos para entrenamiento: 9405


c:\Users\santi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



F1 macro (test): 0.923

Reporte de clasificación (test):
                                              precision    recall  f1-score   support

     Empleo/Salarios/Oportunidades Laborales      0.805     0.872     0.837       109
            Movilidad/Pico y Placa Cartagena      1.000     1.000     1.000        39
  Política/Economía/Medios/Contexto Nacional      0.981     0.923     0.951      1098
   Quejas/Reclamos/Descuentos No Autorizados      0.824     0.959     0.886       146
 Respuestas Automáticas/Community Management      0.878     0.951     0.913       182
SOAT/Fraudes/Carrotanques/Entidades Públicas      0.886     0.937     0.911       207
       Salud/EPS/Retiro y Reforma al Sistema      0.942     0.980     0.961       100

                                    accuracy                          0.932      1881
                                   macro avg      0.902     0.946     0.923      1881
                                weighted avg      0.936     0.932     0.933     

C:\Users\santi\AppData\Local\Temp\ipykernel_13576\2547097012.py:96: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [29]:
# ================================================
# Mostrar ejemplos de tweets por cluster (servicio)
# ================================================

def print_examples(df, cluster_col="topic_km", text_col="text_raw", brand_col="brand_primary", n=5):
    clusters = df[cluster_col].unique()
    for cl in sorted(clusters):
        subset = df[df[cluster_col] == cl]
        print(f"\n=== Cluster {cl} ({subset['topic_name'].iloc[0]}) | n={len(subset)} ===")
        sample = subset.sample(min(n, len(subset)), random_state=42)
        for _, row in sample.iterrows():
            print(f"- [{row[brand_col]}] {row[text_col][:180]}...")

# Llamar a la función (ejemplos de 5 por cluster)
print_examples(df_service, cluster_col="topic_km", text_col="text_raw", brand_col="brand_primary", n=5)



=== Cluster 0 (Cobertura/Condiciones) | n=237 ===
- [mapfre] MAPFRE CRECIÓ 9% Y CONSIGUIÓ UN BENEFICIO DE 24% EN LATINOAMÉRICA DURANTE 2023 Grupo.  A nivel global, MAPFRE obtuvo un resultado neto de 692 millones de euros, lo que representa ...
- [fiduciaria previsora] La Previsora reafirma su liderazgo en el sector asegurador del país gracias a sus múltiples distinciones ...reconocimiento a una vida de trabajo por la protección en el país. En un...
- [] Directv lanza seguro todo riesgo para autos en Colombia: ¿qué beneficios tiene? ...Axa Colpatria. El nuevo seguro está diseñado para ofrecer a los conductores colombianos una alter...
- [] Seis acreedores de la cadena de almacenes de perfumerías y farmacéuticos Fedco denuncian que les incumplió pagos del acuerdo que suma $273.504 millones ...noventa (90) días, por va...
- [mapfre] Conozca las coberturas de seguros para las empresas que ofrecen fiestas navideñas ...; los accidentes personales y el seguro del personal. En específico, en

In [30]:
# ============================================================
# 1) Filtro "Servicio" reforzado (semillas + señales de queja)
#    y exclusión de corporativo/deporte/SOAT informativo
# ============================================================
import re
import numpy as np
import pandas as pd

POSITIVE_SEEDS = re.compile(
    r"(siniestro|indemnizaci[oó]n|reembolso|radicado|reparaci[oó]n|peritaci[oó]n|ajustador|"
    r"autorizaci[oó]n|red_m[eé]dica|cobertura|exclusi[oó]n|preexistenc|deducible|cl[aá]usula|"
    r"condici[oó]n|prima|soat|pqrs|atenci[oó]n_al_cliente|l[ií]nea_de_atenci[oó]n)",
    re.I
)
COMPLAINT = re.compile(
    r"(no\s+(respon|contes|solucion|pagan)|llevo\s+\d+\s+(d[ií]as|meses)|"
    r"por favor|ayuda|necesito|reclamo|queja|radicado|esperando|llamar|"
    r"no han pagado|devoluci[oó]n|ticket|formulario|pqrs)",
    re.I
)
NOT_SERVICE = re.compile(
    r"(colcap|bill[oó]n|dividendo|resultado(?:s)?\s+oficial|beneficio|"
    r"inaugur|ranking|trimestre|semestre|msci|bolsa|utilidad(?:es)?|"
    r"convocatoria|patinaje|selecci[oó]n|campeonato|asamblea|consejo directivo)",
    re.I
)
SOAT_INFO = re.compile(r"(tarifa|valor|listado|c[oó]mo\s+adquirir|recomendacione?s)", re.I)

def looks_like_service(text: str) -> bool:
    t = text or ""
    if NOT_SERVICE.search(t):  # descarta corporativo/deporte/finanzas
        return False
    if "soat" in t.lower() and SOAT_INFO.search(t) and not COMPLAINT.search(t):
        return False  # SOAT informativo sin queja/pedido
    # semillas + (queja/pedido) O semillas fuertes
    has_seed = bool(POSITIVE_SEEDS.search(t))
    has_complaint = bool(COMPLAINT.search(t))
    strong = re.search(r"(siniestro|autorizaci[oó]n|radicado|reparaci[oó]n|red_m[eé]dica)", t, re.I)
    return bool(has_seed and (has_complaint or strong))

# Asegura df_topics (tema-centrado sin corporativo/CM); si no existe, créalo
if 'df_topics' not in globals():
    df_topics = df[~df["flag_corp_news"] & ~df["flag_cm_reply"]].copy()

df_service = df_topics[df_topics["text_topics"].map(looks_like_service)].copy()
print("Docs servicio (nuevo filtro):", len(df_service), " / de", len(df_topics))

# ============================================================
# 2) TF-IDF (1–2) + KMeans en servicio
# ============================================================
import nltk
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans

sw_es = set(stopwords.words('spanish'))

tfidf = TfidfVectorizer(
    ngram_range=(1,2),
    min_df=5,
    max_df=0.6,
    max_features=40000,
    lowercase=False,
    token_pattern=r'(?u)\b\w+\b',
    stop_words=list(sw_es)
)
texts = df_service["text_topics"].fillna("").astype(str).tolist()
X = tfidf.fit_transform(texts)
terms = np.array(tfidf.get_feature_names_out())
print(f"TF-IDF servicio: docs={X.shape[0]}, vocab={X.shape[1]}")

# prueba K (elige el que te deje 3–5 grupos razonables)
for K in [4,5]:
    km_tmp = MiniBatchKMeans(n_clusters=K, random_state=42, batch_size=2048, n_init='auto')
    lbl_tmp = km_tmp.fit_predict(X)
    sizes = pd.Series(lbl_tmp).value_counts().sort_index().tolist()
    print(f"K={K} -> tamaños por cluster:", sizes)

K = 5
km = MiniBatchKMeans(n_clusters=K, random_state=42, batch_size=2048, n_init='auto')
labels = km.fit_predict(X)
df_service["topic_km"] = labels

# Top términos por cluster (para inspección)
centroids = km.cluster_centers_
cluster_top_terms = {}
for k in range(K):
    top_idx = np.argsort(centroids[k])[::-1][:25]
    tops = terms[top_idx]
    cluster_top_terms[k] = ", ".join(tops)
    print(f"\nCluster {k} tops:\n{cluster_top_terms[k]}")

# ============================================================
# 3) Bautizo SIN duplicados (reglas + subreglas)
# ============================================================
RULES_PRIMARY = [
    ("Pagos/Siniestros",  r"(siniestro|indemnizaci[oó]n|reembolso|pago_de_siniestro|radicado|peritaci[oó]n|ajustador|reparaci[oó]n)"),
    ("Atención/Trámite",  r"(atenci[oó]n_al_cliente|l[ií]nea_de_atenci[oó]n|tiempo_de_respuesta|pqrs|queja|reclamo|responder|esperar|llamar|no\s+(respon|contes|solucion))"),
    ("Cobertura/Condiciones", r"(cobertura|exclusi[oó]n|preexistenc|deducible|cl[aá]usula|condici[oó]n)"),
    ("Red/Autorizaciones", r"(red_m[eé]dica|autorizaci[oó]n_de_servicio|autorizaci[oó]n|cl[ií]nica|prestador|cita|remisi[oó]n|taller)"),
    ("Costos/Primas/SOAT", r"(prima|incremento_de_prima|facturaci[oó]n|cobro|d[eé]bito|soat)")
]
RULES_SECONDARY = [
    ("Atención/PQRS", r"(pqrs|radicado|formulario|ticket|canal)"),
    ("Atención/Respuesta", r"(esperar|no\s+(respon|contes)|l[ií]nea_de_atenci[oó]n|llamar)"),
    ("Pagos/Reparación", r"(reparaci[oó]n|taller|peritaje)"),
    ("Cobertura/Preexistencias", r"(preexistenc)"),
]

def name_by_rules(text):
    # primaria
    for name, rx in RULES_PRIMARY:
        if re.search(rx, text, re.I):
            return name
    # secundaria
    for name, rx in RULES_SECONDARY:
        if re.search(rx, text, re.I):
            return name
    return "Tema_mixto"

# asigna nombres; si hay duplicados, desambigua con secundarias / id
names = {}
used = set()
for k in range(K):
    base = name_by_rules(cluster_top_terms[k])
    if base in used:
        # intenta secundaria para diferenciar
        sec = None
        for nm, rx in RULES_SECONDARY:
            if re.search(rx, cluster_top_terms[k], re.I) and nm not in used:
                sec = nm; break
        name = sec or f"{base}#{k}"
    else:
        name = base
    names[k] = name
    used.add(name)

print("\nNombre por cluster:")
for k, n in names.items():
    print(k, "->", n)

df_service["topic_name"] = df_service["topic_km"].map(names)

# ============================================================
# 4) Supervisado (TF-IDF + LogReg) + matriz de confusión
# ============================================================
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score

df_sup = df_service.dropna(subset=["text_topics","topic_name"]).copy()
X_text = df_sup["text_topics"].astype(str)
y      = df_sup["topic_name"].astype(str)

X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X_text, y, df_sup.index, test_size=0.2, random_state=42, stratify=y
)

tfidf_sup = TfidfVectorizer(
    ngram_range=(1,2),
    min_df=5, max_df=0.6, max_features=40000,
    lowercase=False, token_pattern=r'(?u)\b\w+\b',
    stop_words=list(sw_es)
)
Xtr = tfidf_sup.fit_transform(X_train)
Xte = tfidf_sup.transform(X_test)

clf = LogisticRegression(
    max_iter=2000, class_weight="balanced", solver="lbfgs", multi_class="auto"
)
clf.fit(Xtr, y_train)
y_pred = clf.predict(Xte)

print("\nF1 macro (test):", round(f1_score(y_test, y_pred, average="macro"), 3))
print("\nReporte de clasificación (test):")
print(classification_report(y_test, y_pred, digits=3))

labels_sorted = sorted(y.unique())
cm = confusion_matrix(y_test, y_pred, labels=labels_sorted)
cm_df = pd.DataFrame(cm, index=[f"true:{c}" for c in labels_sorted],
                        columns=[f"pred:{c}" for c in labels_sorted])
print("\nMatriz de confusión:")
print(cm_df.to_string())

# Export de test y errores
test_df = df_sup.loc[idx_test].copy()
test_df["y_true"] = y_test.values
test_df["y_pred"] = y_pred
test_df.to_csv("pred_supervisado_temas_test.csv", index=False, encoding="utf-8")
test_df[test_df["y_true"] != test_df["y_pred"]].to_csv("errores_supervisado_temas_test.csv", index=False, encoding="utf-8")
print("\nExportados: pred_supervisado_temas_test.csv, errores_supervisado_temas_test.csv")


Docs servicio (nuevo filtro): 198  / de 9036
TF-IDF servicio: docs=198, vocab=214
K=4 -> tamaños por cluster: [39, 61, 51, 47]
K=5 -> tamaños por cluster: [32, 65, 49, 43, 9]

Cluster 0 tops:
dejar, reparacion, parte, bogota, seguir, decir, solo, avanzar, linea_de_atencion, segun, 01, sistema, grupo_sura, cancelar, preparacion, deber, poner, salud, informacion, proceso, medio, comunicar, asistencia, eps, ano

Cluster 1 tops:
tener, siniestro, soat, responder, pagar, vehiculo, reparacion, correo, acuerdo, dia, dar, suarez, buen, hacer, documento, dato, llevar, 2024, s, radicado, saber, llamar, conductor, respuesta, pago

Cluster 2 tops:
autorizacion, descontar, hacer, dinero, pasar, descuento, llamar, robar, cuenta, servicio, decir, primero, s, mismo, hdi, positiva, financiero, bolivar, persona, ladrón, autorizar, 2, hora, solicitar, tratar

Cluster 3 tops:
siniestro, bolivar, servicio, axa_colpatria, asi, clave, atencion, pagar, compania, eps, medico, scotiabank, digital, dano, scotiab

c:\Users\santi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [39]:
import requests

# URL del lexicón ML-SentiCon en español
url = "https://raw.githubusercontent.com/ITALIC-US/ML-Senticon/main/senticon.es.xml"
output_file = "mlsenticon_es.xml"

print(f"Descargando desde {url}...")
response = requests.get(url)
response.raise_for_status()  # Lanza una excepción si hubo un error en la descarga

# Guarda el contenido XML en disco
with open(output_file, "wb") as f:
    f.write(response.content)

print(f"Lexicón guardado en '{output_file}'")





Descargando desde https://raw.githubusercontent.com/ITALIC-US/ML-Senticon/main/senticon.es.xml...
Lexicón guardado en 'mlsenticon_es.xml'


In [47]:
# ============================================
# ML-SentiCon (ES) end-to-end para sentimiento por marca
# ============================================
import os, re, math, requests, xml.etree.ElementTree as ET
import numpy as np
import pandas as pd

# ---------- RUTAS & FUENTE ----------
# Si ya tienes df_topics, úsalo; si no, usa df
BASE = df_topics.copy() if 'df_topics' in globals() else df.copy()

TEXT_COL  = "text_clean_base"   # puedes cambiar a "text_raw" si prefieres
ID_COL    = "id"
BRAND_COL = "brand_primary"

# Ruta LOCAL preferida del XML (ajústala si la tuya es distinta)
LEX_XML_LOCAL = r"C:\Users\santi\Downloads\Learning\Maestria\Programacion Lenguaje Natural\Proyecto\Entrega 2\mlsenticon_es.xml"

# URL de respaldo por si necesitas descargar el XML (raw GitHub)
LEX_XML_URL = "https://raw.githubusercontent.com/ITALIC-US/ML-Senticon/main/senticon.es.xml"

# ---------- 0) Preparar corpus ----------
base_lex = BASE[[ID_COL, BRAND_COL, TEXT_COL]].copy()
base_lex[TEXT_COL] = base_lex[TEXT_COL].fillna("").astype(str)
base_lex = base_lex[base_lex[TEXT_COL].str.strip().ne("")].copy()
print("Docs a analizar (léxico):", len(base_lex))

# ---------- 1) Obtener el XML de ML-SentiCon (descargar si hace falta) ----------
LEX_XML = LEX_XML_LOCAL
if not os.path.exists(LEX_XML_LOCAL):
    print(f"No se encontró el XML en:\n  {LEX_XML_LOCAL}\nIntento descargarlo…")
    try:
        resp = requests.get(LEX_XML_URL, timeout=60)
        resp.raise_for_status()
        # guarda en la misma carpeta del notebook
        LEX_XML = os.path.join(os.getcwd(), "mlsenticon_es.xml")
        with open(LEX_XML, "wb") as f:
            f.write(resp.content)
        print(f"Descargado ML-SentiCon en: {LEX_XML}")
    except Exception as e:
        raise RuntimeError(f"No pude descargar el lexicón desde {LEX_XML_URL}. Error: {e}")
# ---------- 2) Parsear XML → DataFrame (lemma, polarity) [ROBUSTO] ----------
rows = []
tree = ET.parse(LEX_XML)
root = tree.getroot()

def norm_tag(t):
    return t.split('}')[-1].lower() if '}' in t else t.lower()

POL_KEYS = {"polarity", "priorpolarity", "sentiment", "value", "pol", "score", "strength"}

def extract_polarity_from_attrib(d: dict):
    # Busca una clave de polaridad en atributos; devuelve str o None
    for k, v in d.items():
        if k.lower() in POL_KEYS and v is not None and str(v).strip() != "":
            return str(v).strip()
    return None

for lemma in root.iter():
    if norm_tag(lemma.tag) != "lemma":
        continue

    form = lemma.attrib.get("form") or lemma.attrib.get("word") or (lemma.text or "").strip()
    if not form:
        continue
    form = form.strip().lower()

    # 1) Caso A: la polaridad viene como atributo del propio <lemma ...>
    pol = extract_polarity_from_attrib(lemma.attrib)

    # 2) Caso B: la polaridad viene en hijos tipo <sense ... polarity="...">
    if pol is None:
        for child in list(lemma):
            t = norm_tag(child.tag)
            if t in {"sense", "feat", "polarity"}:
                pol = extract_polarity_from_attrib(child.attrib)
                if pol is None:
                    # algunos usan texto del nodo para el valor
                    txt = (child.text or "").strip()
                    if txt:
                        pol = txt
                if pol is not None:
                    rows.append((form, pol))
        # si ya agregamos por hijos, sigue al próximo lemma
        if pol is not None and not any(r[0] == form for r in rows[-len(list(lemma)) or 0:]):
            # se añadió vía hijos; ya continuamos
            continue

    # 3) Caso C: algún otro nieto (por seguridad)
    if pol is None:
        for child in lemma.iter():
            if child is lemma:
                continue
            pol = extract_polarity_from_attrib(child.attrib)
            if pol:
                rows.append((form, pol))
        if pol:
            continue

    # 4) Caso D: si no se encontró en hijos y sí en el lemma, añádelo
    if pol is not None:
        rows.append((form, pol))

if not rows:
    # Diagnóstico: imprime algunas etiquetas para ver la estructura real
    # (No detiene; pero avisa con ValueError para que lo veas rápido)
    raise ValueError(
        "No se encontraron polaridades en el XML. Es probable que el archivo tenga una estructura distinta.\n"
        "Prueba abrir el XML y buscar nodos como <sense polarity=\"...\"> o <feat name=\"polarity\" value=\"...\">."
    )

lex_df = pd.DataFrame(rows, columns=["lemma", "polarity"]).dropna()
print(f"Lexicón (crudo) extraído: {len(lex_df):,} filas")

# Normaliza y deduplica (si un lemma tiene varias senses, promedia)
lex_df["lemma"] = lex_df["lemma"].astype(str).str.strip().str.lower()

def _map_polarity(x):
    x = str(x).strip().lower()
    if x in ("positive","pos","+","1","true"): return 1.0
    if x in ("negative","neg","-","-1","false"): return -1.0
    if x in ("neutral","neu","0"): return 0.0
    try:
        return float(x)
    except:
        return 0.0

lex_df["pol"] = lex_df["polarity"].map(_map_polarity).astype(float)
lex_df = lex_df.groupby("lemma", as_index=False)["pol"].mean()
print(f"Lexicón (normalizado): {len(lex_df):,} lemas únicos")


# ---------- 3) Normalizar polaridad a float [-1, 1] y construir diccionario ----------
def _map_polarity(x):
    x = str(x).strip().lower()
    if x in ("positive","pos","+","1","true"): return 1.0
    if x in ("negative","neg","-","-1","false"): return -1.0
    if x in ("neutral","neu","0"): return 0.0
    try:
        return float(x)
    except:
        return 0.0

# Caso A: si aún existe 'polarity', la convertimos a 'pol'
if "polarity" in lex_df.columns:
    lex_df["pol"] = lex_df["polarity"].map(_map_polarity).astype(float)
    # opcional: quitar 'polarity' para evitar confusiones posteriores
    # lex_df = lex_df.drop(columns=["polarity"])

# Caso B: si ya NO existe 'polarity', asumimos que 'pol' ya está construida en el paso robusto
if "pol" not in lex_df.columns:
    raise ValueError("No encuentro la columna 'pol' en lex_df. Revisa el bloque de parseo robusto.")

# Asegura un único valor por lemma (si no lo hiciste ya)
lex_df = (lex_df.groupby("lemma", as_index=False)["pol"]
                 .mean())

# Construye el diccionario final
lexicon = dict(zip(lex_df["lemma"], lex_df["pol"]))
print(f"Vocabulario léxico final: {len(lexicon):,} términos")


# (Opcional) Ajustes de dominio: añade/ajusta términos propios del sector
lexicon.update({
    # señales negativas típicas del dominio
    "queja": -0.6, "reclamo": -0.6, "radicado": -0.2,
    "siniestro": -0.1,   # neutral/levemente negativo (ajusta a gusto)
    "indemnización": 0.2, "indemnizacion": 0.2,
    "no": 0.0,  # negador se maneja aparte (no debe tener score propio)
    "soat": 0.0, "pqrs": -0.4
})

# ---------- 4) Tokenización + Negación + Intensificadores ----------
TOKEN_RE = re.compile(r"[A-Za-zÁÉÍÓÚÜÑáéíóúüñ0-9_]+", re.UNICODE)

NEGATORS = {"no","nunca","jamás","jamas","ninguno","ninguna","nadie","tampoco","sin"}
INTENSIFIERS = {
    "muy":1.5,"re":1.5,"super":1.5,"súper":1.5,"tan":1.2,"bastante":1.3,"demasiado":1.3,
    "un_poco":0.7,"poco":0.7,"algo":0.8,"apenas":0.8
}
NEG_WINDOW = 3  # número de tokens siguientes a negar si tienen score

def tokenize_simple(text: str):
    return TOKEN_RE.findall((text or "").lower())

def sentiment_score(tokens):
    score = 0.0
    i = 0
    while i < len(tokens):
        tok = tokens[i]
        mult = INTENSIFIERS.get(tok, 1.0)

        # negación: invierte los próximos NEG_WINDOW con score
        if tok in NEGATORS:
            j = i + 1
            flips = 0
            while j < len(tokens) and flips < NEG_WINDOW:
                pol = lexicon.get(tokens[j], 0.0)
                if pol != 0.0:
                    score += -pol
                    flips += 1
                j += 1
            i += 1
            continue

        pol = lexicon.get(tok, 0.0)
        if pol != 0.0:
            score += pol * mult
        i += 1
    return score

def label_from_score(s, pos_th=0.25, neg_th=-0.25):
    if s >= pos_th:
        return "pos"
    if s <= neg_th:
        return "neg"
    return "neu"

# ---------- 5) Puntuar todo el corpus ----------
scores, labels = [], []
for txt in base_lex[TEXT_COL].tolist():
    toks = tokenize_simple(txt)
    sc = sentiment_score(toks)
    scores.append(sc)
    labels.append(label_from_score(sc))

base_lex["lex_score"] = scores
base_lex["lex_label"] = labels

print("\nDistribución etiquetas (léxico):")
print(base_lex["lex_label"].value_counts().to_string())

# ---------- 6) Agregados por marca ----------
agg_counts_lex = (
    base_lex.pivot_table(index=BRAND_COL, columns="lex_label", values=ID_COL, aggfunc="count", fill_value=0)
            .rename_axis(None, axis=1)
)
agg_counts_lex["total"] = agg_counts_lex.sum(axis=1)
agg_props_lex = agg_counts_lex.div(agg_counts_lex["total"], axis=0)

print("\nConteos por marca (léxico) — top 15:")
print(agg_counts_lex.sort_values("total", ascending=False).head(15).to_string())
print("\nProporciones por marca (léxico) — top 15:")
print(agg_props_lex.sort_values("total", ascending=False).head(15).round(3).to_string())

# ---------- 7) (Opcional) Comparar con pysentimiento si ya lo corriste ----------
# Si tienes un DataFrame "base" de tu bloque de pysentimiento con 'sent_label'
if 'base' in globals() and isinstance(base, pd.DataFrame) and "sent_label" in base.columns:
    try:
        comp = base[[ID_COL, BRAND_COL, TEXT_COL, "sent_label"]].merge(
            base_lex[[ID_COL, "lex_label","lex_score"]], on=ID_COL, how="inner"
        )
        comp.to_csv("comparacion_lexico_vs_pysentimiento.csv", index=False, encoding="utf-8")
        print("\nExportado: comparacion_lexico_vs_pysentimiento.csv")
    except Exception as e:
        print("Aviso: no se pudo crear comparacion_lexico_vs_pysentimiento.csv:", e)

# ---------- 8) Exports ----------
base_lex.to_csv("sentimiento_pred_mlsenticon.csv", index=False, encoding="utf-8")
agg_counts_lex.to_csv("sentimiento_por_marca_conteos_mlsenticon.csv", encoding="utf-8")
agg_props_lex.to_csv("sentimiento_por_marca_proporciones_mlsenticon.csv", encoding="utf-8")
print("\nExportados (léxico):")
print("- sentimiento_pred_mlsenticon.csv")
print("- sentimiento_por_marca_conteos_mlsenticon.csv")
print("- sentimiento_por_marca_proporciones_mlsenticon.csv")




Docs a analizar (léxico): 9036
Lexicón (crudo) extraído: 11,542 filas
Lexicón (normalizado): 11,342 lemas únicos
Vocabulario léxico final: 11,342 términos

Distribución etiquetas (léxico):
lex_label
pos    4766
neu    2809
neg    1461

Conteos por marca (léxico) — top 15:
                       neg   neu   pos  total
brand_primary                                
                      1040  1898  3023   5961
fiduciaria previsora   145   284   466    895
seguros bolivar         83   213   536    832
sura                    86   178   315    579
mapfre                  61   116   208    385
allianz                 18    35    94    147
liberty seguros         15    52    69    136
hdi seguros              5    24    33     62
axa colpatria            3     2    16     21
porvenir                 3     5     3     11
zurich                   2     2     3      7

Proporciones por marca (léxico) — top 15:
                        neg    neu    pos  total
brand_primary                        

In [51]:
# ============================================================
# INFORME: Ejemplos, gráficas y métricas de calidad (sentimiento)
# ============================================================
import os, re, math, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_recall_fscore_support, cohen_kappa_score

random.seed(42)
np.random.seed(42)

# ----------------------------
# 0) ENTRADAS / ALIAS  (robusto)
# ----------------------------
# DF_LEX: debe tener [id, brand_primary, text_raw/text_clean_base, lex_label, lex_score]
DF_LEX = base_lex.copy()

# Intentamos localizar el DataFrame de pysentimiento (opcional):
DF_PY = None
if 'base' in globals() and isinstance(base, pd.DataFrame) and ('sent_label' in base.columns):
    DF_PY = base.copy()
elif 'df_pysent' in globals() and isinstance(df_pysent, pd.DataFrame) and ('sent_label' in df_pysent.columns):
    DF_PY = df_pysent.copy()
# si no existe, DF_PY se queda en None y se saltan las métricas de comparación

# Resolver columna de texto de manera segura
TEXT_COL_CANDIDATES = ["text_raw", "text_clean_base", "text_topics"]
TEXT_COL = next((c for c in TEXT_COL_CANDIDATES if c in DF_LEX.columns), None)
assert TEXT_COL is not None, f"No encuentro columna de texto en DF_LEX. Probé: {TEXT_COL_CANDIDATES}"

BRAND_COL = "brand_primary" if "brand_primary" in DF_LEX.columns else "brand"
ID_COL    = "id" if "id" in DF_LEX.columns else DF_LEX.columns[0]  # fallback prudente

LEX_LABEL = "lex_label"
LEX_SCORE = "lex_score"
PY_LABEL  = "sent_label"  # en DF_PY
TOPN_BRANDS = 12
OUT_DIR = "informe_sentimiento"
os.makedirs(OUT_DIR, exist_ok=True)

# Validaciones mínimas
for col in [ID_COL, BRAND_COL, TEXT_COL, LEX_LABEL, LEX_SCORE]:
    assert col in DF_LEX.columns, f"Falta columna {col} en DF_LEX"


# ----------------------------
# 1) AGREGADOS BÁSICOS
# ----------------------------
def agregados_por_marca(df, label_col):
    counts = (df
              .pivot_table(index=BRAND_COL, columns=label_col, values=ID_COL, aggfunc="count", fill_value=0)
              .rename_axis(None, axis=1))
    counts["total"] = counts.sum(axis=1)
    props = counts.div(counts["total"], axis=0)
    return counts, props

lex_counts, lex_props = agregados_por_marca(DF_LEX, LEX_LABEL)
lex_counts.to_csv(os.path.join(OUT_DIR, "conteos_por_marca_lex.csv"), encoding="utf-8")
lex_props.to_csv(os.path.join(OUT_DIR, "proporciones_por_marca_lex.csv"), encoding="utf-8")

print("\n[LEX] Conteos por marca (top):")
print(lex_counts.sort_values("total", ascending=False).head(15).to_string())
print("\n[LEX] Proporciones por marca (top):")
print(lex_props.sort_values("total", ascending=False).head(15).round(3).to_string())

# ----------------------------
# 2) EJEMPLOS (muestras)
# ----------------------------
def ejemplos(df, brand=None, label=None, n=10):
    tmp = df.copy()
    if brand is not None:
        tmp = tmp[tmp[BRAND_COL].astype(str).str.lower() == str(brand).lower()]
    if label is not None:
        tmp = tmp[tmp[LEX_LABEL] == label]
    n = min(n, len(tmp))
    if n == 0:
        return pd.DataFrame(columns=[ID_COL, BRAND_COL, LEX_LABEL, LEX_SCORE, TEXT_COL])
    asc = (label == "neg") if label is not None else False
    return (tmp[[ID_COL, BRAND_COL, LEX_LABEL, LEX_SCORE, TEXT_COL]]
            .sample(n, random_state=42)
            .sort_values(LEX_SCORE, ascending=asc))


# Exporta ejemplos por clase (global) y por top marcas
ej_pos = ejemplos(DF_LEX, label="pos", n=30)
ej_neu = ejemplos(DF_LEX, label="neu", n=30)
ej_neg = ejemplos(DF_LEX, label="neg", n=30)
ej_pos.to_csv(os.path.join(OUT_DIR, "ejemplos_pos.csv"), index=False, encoding="utf-8")
ej_neu.to_csv(os.path.join(OUT_DIR, "ejemplos_neu.csv"), index=False, encoding="utf-8")
ej_neg.to_csv(os.path.join(OUT_DIR, "ejemplos_neg.csv"), index=False, encoding="utf-8")

top_brands = lex_counts.sort_values("total", ascending=False).head(TOPN_BRANDS).index.tolist()
with pd.ExcelWriter(os.path.join(OUT_DIR, "ejemplos_por_marca.xlsx"), engine="xlsxwriter") as wr:
    for b in top_brands:
        sub = pd.concat([
            ejemplos(DF_LEX, brand=b, label="neg", n=15),
            ejemplos(DF_LEX, brand=b, label="neu", n=10),
            ejemplos(DF_LEX, brand=b, label="pos", n=10)
        ], axis=0)
        sub.to_excel(wr, sheet_name=(b[:30] or "sinmarca"), index=False)

# ----------------------------
# 3) GRÁFICAS (matplotlib, sin seaborn)
# ----------------------------
# Proporción de sentimiento por marca (Top) — apilada
plt.figure(figsize=(12, 5))
lex_props_sorted = lex_props.sort_values("total", ascending=False).head(TOPN_BRANDS)
order = [c for c in ["neg","neu","pos"] if c in lex_props_sorted.columns]
bottom = np.zeros(len(lex_props_sorted))
for cls in order:
    vals = lex_props_sorted[cls].values
    plt.bar(lex_props_sorted.index.astype(str), vals, bottom=bottom, label=cls)
    bottom = bottom + vals
plt.xticks(rotation=45, ha="right")
plt.title("Proporción de sentimiento por marca (Top marcas) — Lexicón")
plt.ylabel("Proporción")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "stacked_proporcion_sentimiento_por_marca_lex.png"), dpi=160)
plt.close()


plt.figure(figsize=(12, 4))
lex_counts_sorted = lex_counts.sort_values("total", ascending=False).head(TOPN_BRANDS)
plt.bar(lex_counts_sorted.index.astype(str), lex_counts_sorted["total"].values)
plt.xticks(rotation=45, ha="right")
plt.title("Volumen por marca (Top marcas)")
plt.ylabel("Docs")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "barras_volumen_por_marca.png"), dpi=160)
plt.close()

# Distribución global
plt.figure(figsize=(6,4))
dist_global = DF_LEX[LEX_LABEL].value_counts()
plt.bar(dist_global.index.astype(str), dist_global.values)
plt.title("Distribución global de sentimiento (Lexicón)")
plt.ylabel("Docs")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "distribucion_global_lex.png"), dpi=160)
plt.close()
# ----------------------------
# 4) MÉTRICAS DE CALIDAD (sin gold): acuerdo Lex vs Pysentimiento
# ----------------------------
if isinstance(DF_PY, pd.DataFrame) and (PY_LABEL in DF_PY.columns):
    cmp_df = (DF_LEX[[ID_COL, LEX_LABEL]]
              .merge(DF_PY[[ID_COL, PY_LABEL]], on=ID_COL, how="inner")
              .dropna(subset=[LEX_LABEL, PY_LABEL]))
    print("\nTamaño de intersección Lex vs PySent:", len(cmp_df))

    labels = sorted(list(set(cmp_df[LEX_LABEL].unique()) | set(cmp_df[PY_LABEL].unique())))
    cm = confusion_matrix(cmp_df[PY_LABEL], cmp_df[LEX_LABEL], labels=labels)
    print("\nMatriz de confusión (Pysent → filas, Lex → columnas):")
    cm_df = pd.DataFrame(cm, index=[f"py:{l}" for l in labels], columns=[f"lex:{l}" for l in labels])
    print(cm_df.to_string())

    print("\nReporte vs. pysentimiento (macro):")
    print(classification_report(cmp_df[PY_LABEL], cmp_df[LEX_LABEL], labels=labels, digits=3))

    kappa = cohen_kappa_score(cmp_df[PY_LABEL], cmp_df[LEX_LABEL])
    print("\nCohen's kappa (Lex vs PySent):", round(kappa, 3))

    plt.figure(figsize=(5,4))
    plt.imshow(cm, interpolation='nearest')
    plt.title("Confusión PySent vs Lex")
    plt.xticks(range(len(labels)), labels)
    plt.yticks(range(len(labels)), labels)
    plt.xlabel("Lex (pred)")
    plt.ylabel("PySent (ref)")
    for i in range(len(labels)):
        for j in range(len(labels)):
            plt.text(j, i, cm[i, j], ha="center", va="center")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "confusion_pysent_vs_lex.png"), dpi=160)
    plt.close()
else:
    print("\n[AVISO] No se encontró DF_PY válido con columna 'sent_label'. Saltando comparación Lex vs PySent.")


# ----------------------------
# 5) DRIVERS / TÉRMINOS CARACTERÍSTICOS
#    (n-gramas más frecuentes por clase y por marca)
# ----------------------------
TOKEN_RE = re.compile(r"[A-Za-zÁÉÍÓÚÜÑáéíóúüñ0-9]+", re.UNICODE)
def ngrams(tokens, n=2):
    return ["_".join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def top_terms(df, label=None, brand=None, n_top=25, ngram=(1,2)):
    tmp = df.copy()
    if label is not None:
        tmp = tmp[tmp[LEX_LABEL] == label]
    if brand is not None:
        tmp = tmp[tmp[BRAND_COL] == brand]
    bag = []
    for txt in tmp[TEXT_COL].astype(str):
        toks = [t.lower() for t in TOKEN_RE.findall(txt)]
        if 1 in ngram:
            bag.extend(toks)
        if 2 in ngram:
            bag.extend(ngrams(toks, 2))
    return Counter(bag).most_common(n_top)

# Global drivers por clase
drivers = {}
for lab in ["neg","neu","pos"]:
    drivers[lab] = top_terms(DF_LEX, label=lab, n_top=40, ngram=(1,2))
    pd.DataFrame(drivers[lab], columns=["term","freq"]).to_csv(
        os.path.join(OUT_DIR, f"drivers_{lab}_global.csv"), index=False, encoding="utf-8"
    )

# Drivers por marca (solo top marcas)
for b in top_brands:
    for lab in ["neg","pos"]:
        dd = top_terms(DF_LEX, label=lab, brand=b, n_top=30, ngram=(1,2))
        pd.DataFrame(dd, columns=["term","freq"]).to_csv(
            os.path.join(OUT_DIR, f"drivers_{lab}_{b[:30]}.csv"), index=False, encoding="utf-8"
        )

# ----------------------------
# 6) ANOTACIÓN HUMANA (estratificada) + EVALUACIÓN
# ----------------------------
def sample_for_annotation(df, per_label=60, per_brand=0):
    # Estratifica por sentimiento y, opcionalmente, por marca
    frames = []
    labels = df[LEX_LABEL].dropna().unique().tolist()
    if per_brand > 0:
        brands = df[BRAND_COL].value_counts().index.tolist()
        for b in brands[:TOPN_BRANDS]:
            for lab in labels:
                subset = df[(df[BRAND_COL]==b) & (df[LEX_LABEL]==lab)]
                k = min(per_brand, len(subset))
                if k>0:
                    frames.append(subset.sample(k, random_state=42))
    else:
        for lab in labels:
            subset = df[df[LEX_LABEL]==lab]
            k = min(per_label, len(subset))
            if k>0:
                frames.append(subset.sample(k, random_state=42))
    if frames:
        return pd.concat(frames, axis=0).drop_duplicates(subset=[ID_COL])
    return pd.DataFrame(columns=df.columns)

ann = sample_for_annotation(DF_LEX, per_label=80, per_brand=0)
ann = ann[[ID_COL, BRAND_COL, TEXT_COL, LEX_LABEL, LEX_SCORE]].copy()
ann["gold_label"] = ""  # para que un humano complete {pos,neu,neg}
ann_path = os.path.join(OUT_DIR, "para_anotar_sentimiento.csv")
ann.to_csv(ann_path, index=False, encoding="utf-8")
print(f"\nArchivo de ANOTACIÓN creado: {ann_path}")
print("→ Pide a un anotador que llene la columna 'gold_label' con {pos, neu, neg}")

# Si ya tienes anotado un archivo gold, carga y evalúa
GOLD_IN = os.path.join(OUT_DIR, "para_anotar_sentimiento_gold.csv")  # renombra a este cuando lo tengas completo
if os.path.exists(GOLD_IN):
    gold = pd.read_csv(GOLD_IN)
    gold = gold.dropna(subset=["gold_label"])
    # une con tus predicciones actuales (lex)
    eval_df = gold.merge(DF_LEX[[ID_COL, LEX_LABEL]], on=ID_COL, how="left", suffixes=("_gold","_lex"))
    eval_df = eval_df.dropna(subset=[LEX_LABEL])
    print("\nEvaluación vs. GOLD:")
    print(classification_report(eval_df["gold_label"], eval_df[LEX_LABEL], digits=3))
    # Matriz y kappa
    labs_eval = ["neg","neu","pos"]
    cm_gold = confusion_matrix(eval_df["gold_label"], eval_df[LEX_LABEL], labels=labs_eval)
    print("\nMatriz de confusión (GOLD vs LEX):")
    print(pd.DataFrame(cm_gold, index=[f"gold:{l}" for l in labs_eval], columns=[f"lex:{l}" for l in labs_eval]).to_string())
    print("Kappa (GOLD vs LEX):", round(cohen_kappa_score(eval_df["gold_label"], eval_df[LEX_LABEL]), 3))
    # Grabar reporte
    with open(os.path.join(OUT_DIR, "reporte_gold_vs_lex.txt"), "w", encoding="utf-8") as f:
        f.write(classification_report(eval_df["gold_label"], eval_df[LEX_LABEL], digits=3))



[LEX] Conteos por marca (top):
                       neg   neu   pos  total
brand_primary                                
                      1040  1898  3023   5961
fiduciaria previsora   145   284   466    895
seguros bolivar         83   213   536    832
sura                    86   178   315    579
mapfre                  61   116   208    385
allianz                 18    35    94    147
liberty seguros         15    52    69    136
hdi seguros              5    24    33     62
axa colpatria            3     2    16     21
porvenir                 3     5     3     11
zurich                   2     2     3      7

[LEX] Proporciones por marca (top):
                        neg    neu    pos  total
brand_primary                                   
                      0.174  0.318  0.507    1.0
allianz               0.122  0.238  0.639    1.0
axa colpatria         0.143  0.095  0.762    1.0
fiduciaria previsora  0.162  0.317  0.521    1.0
hdi seguros           0.081  0.387  0.5

In [3]:
import pandas as pd
# Corpus base con métricas léxicas y marcas
df = pd.read_csv("corpus_medido.csv")

# Tópicos globales (KMeans general)
df_topics = pd.read_csv("corpus_con_temas_kmeans.csv")

# Tópicos de servicio
df_service = pd.read_csv("corpus_servicio_temas_kmeans.csv")

# Sentimiento (ML-SentiCon)
base_lex = pd.read_csv("sentimiento_pred_mlsenticon.csv")

def build_final_dataset(df_base, df_topics=None, df_service=None, df_sent=None):
    import pandas as pd
    df_final = df_base.copy()
    if df_topics is not None and all(c in df_topics.columns for c in ["topic_km", "topic_name"]):
        df_final.loc[df_topics.index, "topic_km_global"] = df_topics["topic_km"]
        df_final.loc[df_topics.index, "topic_name_global"] = df_topics["topic_name"]
    if df_service is not None and all(c in df_service.columns for c in ["topic_km", "topic_name"]):
        df_final.loc[df_service.index, "topic_km_service"] = df_service["topic_km"]
        df_final.loc[df_service.index, "topic_name_service"] = df_service["topic_name"]
    if df_sent is not None and all(c in df_sent.columns for c in ["lex_label", "lex_score"]):
        df_final = df_final.merge(df_sent[["id", "lex_label", "lex_score"]], on="id", how="left")
    cols_order = [
        "id", "Date", "brand_primary", "brand_list", "text_raw", "text_clean_base",
        "flag_corp_news", "flag_cm_reply", "n_chars_raw", "n_tokens_lemma",
        "n_tokens_topics", "ttr_lemma", "hapax_lemma",
        "topic_km_global", "topic_name_global",
        "topic_km_service", "topic_name_service",
        "lex_label", "lex_score"
    ]
    cols_order = [c for c in cols_order if c in df_final.columns]
    df_final = df_final[cols_order + [c for c in df_final.columns if c not in cols_order]]
    print(f"Dataset final construido: {df_final.shape[0]} registros, {df_final.shape[1]} columnas")
    return df_final

df_final = build_final_dataset(
    df_base=df,
    df_topics=df_topics,
    df_service=df_service,
    df_sent=base_lex
)

df_final.to_csv(
    "dataset_final_completo.csv",
    sep=";",               # separador punto y coma
    index=False,           # no incluir el índice
    encoding="utf-8",      # codificación estándar
    quoting=1,             # 1 = csv.QUOTE_ALL (cita todas las celdas)
    quotechar='"',         # usa comillas dobles para encerrar texto
    lineterminator="\n"    # ✅ nombre correcto del argumento
)



Dataset final construido: 15469 registros, 23 columnas


In [7]:
df_final.columns

Index(['id', 'Date', 'brand_primary', 'brand_list', 'text_raw',
       'text_clean_base', 'n_chars_raw', 'n_tokens_lemma', 'n_tokens_topics',
       'ttr_lemma', 'hapax_lemma', 'topic_km_global', 'topic_name_global',
       'topic_km_service', 'topic_name_service', 'lex_label', 'lex_score',
       'Domain', 'Url', 'text_blocked', 'text_lemma', 'text_stem',
       'text_topics'],
      dtype='object')

In [8]:
df['brand_primary'].value_counts()

brand_primary
sura                  1264
seguros bolivar       1036
mapfre                 605
allianz                310
axa colpatria          237
liberty seguros        143
hdi seguros             90
seguros del estado      34
porvenir                25
zurich                   7
Name: count, dtype: int64